In [1]:
from pyomo.environ import *

In [2]:
def solve_pyomo(para):
    p_ei  = para['pei']
    p_er  = para['per']
    p_ea  = 1-p_ei-p_er
    p_ai  = 0.10714
    p_ar  = 0.04545
    p_irr = 1/8
    alpha  = 0.6
    beta_1 = para['beta1']
    beta_2 = para['beta2']
    cost_tr = 0.0977
    p_id  = 1.1/1000
    p_idd = 0.1221/1000
    p_srr = 0.7/3
    S = 100000
    E = 20
    A = 0
    I = 0
    R = 0
    D = 0
    N = S+E+A+I+R+D
    K = 50
    w_max  = 1.0
    tr_max = 1.0
    v_max = 0.2/14
    M = 3.6e3
    
    m = ConcreteModel()
    m.k = RangeSet(0, K)
    m.t = RangeSet(0, K-1)
    m.N = N
    m.tri = Var(m.k, bounds=(0, 1)) 
    m.wi = Var(m.k, bounds=(0, 1))  
    m.v = Var(m.k, bounds=(0, 1))
    m.aq = Var(m.k, bounds=(0, 1))
    m.iq = Var(m.k, bounds=(0, 1))
    m.x_s = Var(m.k, domain = NonNegativeReals)
    m.x_e = Var(m.k, domain = NonNegativeReals)
    m.x_a = Var(m.k, domain = NonNegativeReals)
    m.x_i = Var(m.k, domain = NonNegativeReals)
    m.x_r = Var(m.k, domain = NonNegativeReals)
    m.x_d = Var(m.k, domain = NonNegativeReals)
    
    m.xs_c = Constraint(m.t, rule=lambda m, k: m.x_s[k+1] == m.x_s[k]
                    -(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                    *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                    -p_srr*m.v[k]*v_max*m.x_s[k])

    m.xe_c = Constraint(m.t, rule=lambda m, k: m.x_e[k+1] == m.x_e[k]
                        +alpha*(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        -p_ei*m.x_e[k]
                        -p_er*m.x_e[k]
                        -m.wi[k]*p_ea*m.x_e[k])
                     
    m.xa_c = Constraint(m.t, rule=lambda m, k: m.x_a[k+1] == m.x_a[k]
                        +m.wi[k]*p_ea*m.x_e[k]
                        -p_ai*m.x_a[k]
                        -p_ar*m.x_a[k])
                                        
    m.xi_c = Constraint(m.t, rule=lambda m, k: m.x_i[k+1] == m.x_i[k]
                        +(1-alpha)*(beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        +p_ei*m.x_e[k]
                        +p_ai*m.x_a[k]
                        -m.tri[k]*p_irr*m.x_i[k]
                        -(1-m.tri[k])*m.x_i[k]*p_id
                        -m.tri[k]*m.x_i[k]*p_idd)
                                                                                       
    m.xr_c = Constraint(m.t, rule=lambda m, k: m.x_r[k+1] == m.x_r[k]
                        +m.tri[k]*p_irr*m.x_i[k]
                        +p_er*m.x_e[k]
                        +p_ar*m.x_a[k]
                        +p_srr*m.v[k]*v_max*m.x_s[k])

    m.xd_c = Constraint(m.t, rule=lambda m, k: m.x_d[k+1] == m.x_d[k]
                        +(1-m.tri[k])*m.x_i[k]*p_id
                        +m.tri[k]*m.x_i[k]*p_idd)
    
    m.pc = ConstraintList()
    m.pc.add(m.x_s[0]==S)
    m.pc.add(m.x_e[0]==E)
    m.pc.add(m.x_a[0]==A)
    m.pc.add(m.x_i[0]==I)
    m.pc.add(m.x_r[0]==R)
    m.pc.add(m.x_d[0]==D)

    gamma = 1
    m.sumcost = sum(m.wi[k]*(0.000369*(m.wi[k]*(m.x_s[k]+m.x_a[k]+m.x_e[k]+m.x_r[k])/m.N)
                         *(m.wi[k]*(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_r[k])/m.N) + 0.001057)
                         *(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_r[k])
                    +0.0977*m.tri[k]*m.x_i[k]+m.x_s[k]
                    *m.v[k]*v_max*0.07 + m.x_i[k]*m.iq[k]*0.02 + m.x_a[k]*m.aq[k]*0.02 for k in m.t) 
    m.budget_c = Constraint(expr = m.sumcost <= M)

    m.suminfected = sum(2.6 * (beta_1*(1-m.iq[k])*m.x_i[k]+beta_2*(m.x_e[k]+(1-m.aq[k])*m.x_a[k]))
                        *(1-m.v[k]*v_max)*m.x_s[k]/(m.x_s[k]+m.x_e[k]+m.x_a[k]+m.x_i[k]+m.x_r[k])
                        +(m.x_d[k+1] - m.x_d[k]) * 60 for k in m.t)
    m.obj = Objective(expr = m.suminfected, sense = minimize)
    solver = SolverFactory('ipopt')
    solver.options['max_iter']= 50000
    try:
        solver.solve(m,tee=True)
    except:
        print()

In [3]:
import pickle
interval = 0.1
para_truth = pickle.load(open('para_truth' + str(round(interval * 100)), 'rb'))

In [4]:
solve_pyomo(para_truth[0])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 187r 1.0619263e+04 9.74e+04 7.99e+02  -3.0 1.84e+02  -1.8 5.00e-02 2.70e-01f  1
 188r 1.0713664e+04 9.74e+04 8.54e+02  -3.0 4.49e+02    -  3.32e-02 4.08e-02f  1
 189r 1.0716851e+04 9.74e+04 2.12e+02  -3.0 1.23e+01  -0.5 1.00e+00 5.76e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.0770315e+04 9.74e+04 2.74e+01  -3.0 3.43e+01  -0.9 1.57e-02 3.42e-01f  1
 191r 1.0771447e+04 9.74e+04 4.80e+02  -3.0 8.38e+01  -1.4 2.90e-01 2.87e-03f  1
 192r 1.1036357e+04 9.73e+04 4.10e+02  -3.0 4.20e+02    -  3.64e-02 1.23e-01f  1
 193r 1.1154457e+04 9.73e+04 1.52e+02  -3.0 3.68e+02    -  8.74e-02 6.24e-02f  1
 194r 1.1277408e+04 9.73e+04 4.28e+02  -3.0 3.03e+01  -1.0 3.19e-01 8.83e-01f  1
 195r 1.1479333e+04 9.72e+04 7.45e+02  -3.0 3.24e+02    -  7.83e-02 1.22e-01f  1
 196r 1.1479556e+04 9.72e+04 5.12e+02  -3.0 2.84e+02    -  3.01e-02 1.54e-04f  1
 197r 1.1486891e+04 9.72e+04 7.35e+02  -3.0 2.82e+02    -  8.34e-02 5.49e-03f  1
 198r 1.1534947e+04 9.72e+04

 481r 3.4362053e+04 9.00e+04 9.99e+02   5.0 0.00e+00   3.0 0.00e+00 8.81e-09R  2
 482r 3.4311730e+04 9.00e+04 9.99e+02   5.0 7.36e+08    -  4.66e-09 5.83e-08f  1
 483r 3.4296919e+04 9.00e+04 1.66e+04   2.9 3.98e+05    -  7.43e-03 2.96e-05f  1
 484r 3.4890420e+04 8.98e+04 1.66e+04   2.9 1.80e+05    -  1.66e-03 1.15e-03f  1
 485r 3.5570116e+04 8.96e+04 1.70e+04   2.9 1.81e+05    -  3.67e-03 1.35e-03f  1
 486r 3.7282281e+04 8.89e+04 2.38e+04   2.9 1.60e+05    -  1.02e-02 4.20e-03f  1
 487r 3.9759740e+04 8.79e+04 3.35e+04   2.9 2.35e+05    -  1.16e-02 4.89e-03f  1
 488r 4.0357708e+04 8.76e+04 3.85e+04   2.2 4.96e+05    -  3.02e-03 6.79e-04f  1
 489r 4.1759987e+04 8.67e+04 3.59e+04   2.2 9.80e+05    -  3.65e-05 1.19e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 490r 4.1949936e+04 8.66e+04 4.02e+04   2.2 8.94e+05    -  1.75e-03 1.26e-04f  1
 491r 4.4054896e+04 8.53e+04 3.69e+04   2.2 1.37e+06    -  3.02e-05 1.10e-03f  1
 492r 4.4442991e+04 8.50e+04

 621  1.6505336e+05 1.17e+03 7.94e+05  -1.0 2.03e+03  -1.6 1.98e-03 6.51e-03h  1
 622  1.6505579e+05 1.17e+03 9.17e+05  -1.0 1.79e+03  -1.2 2.23e-02 4.78e-04h  1
 623  1.6507808e+05 1.17e+03 9.10e+05  -1.0 2.41e+03  -1.6 5.03e-04 4.60e-03h  1
 624  1.6507888e+05 1.17e+03 9.33e+05  -1.0 1.96e+03  -1.2 7.10e-03 1.61e-04h  1
 625  1.6511623e+05 1.16e+03 9.46e+05  -1.0 1.96e+03  -1.7 2.19e-02 7.69e-03h  1
 626  1.6512207e+05 1.16e+03 9.54e+05  -1.0 1.78e+03  -1.3 3.12e-03 1.17e-03h  1
 627  1.6515951e+05 1.15e+03 9.44e+05  -1.0 1.76e+03  -1.7 5.28e-03 7.64e-03h  1
 628  1.6516024e+05 1.15e+03 1.01e+06  -1.0 1.63e+03  -1.3 1.42e-02 1.49e-04h  1
 629  1.6518342e+05 1.14e+03 1.00e+06  -1.0 1.78e+03  -1.8 5.47e-03 5.10e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 630  1.6518464e+05 1.14e+03 1.08e+06  -1.0 1.65e+03  -1.4 1.77e-02 2.51e-04h  1
 631  1.6521416e+05 1.14e+03 1.04e+06  -1.0 1.51e+03  -0.9 3.41e-03 5.87e-03h  1
 632  1.6525341e+05 1.13e+03

 741  1.6751926e+05 2.10e+02 2.29e+06  -1.0 9.24e+02  -2.9 5.14e-03 1.45e-01f  1
 742  1.6751812e+05 2.07e+02 2.17e+06  -1.0 8.47e+02  -2.4 3.50e-02 1.25e-02h  1
 743  1.6752594e+05 2.04e+02 8.10e+06  -1.0 6.17e+02  -1.1 2.34e-01 1.44e-02h  1
 744  1.6753178e+05 2.01e+02 8.02e+06  -1.0 7.43e+02  -1.6 4.16e-02 1.41e-02h  1
 745  1.6754942e+05 1.86e+02 6.84e+06  -1.0 7.87e+02  -2.1 1.51e-01 7.52e-02h  1
 746  1.6754268e+05 1.81e+02 4.70e+06  -1.0 7.85e+02  -2.5 2.23e-01 2.97e-02f  1
 747  1.6754136e+05 1.80e+02 4.55e+06  -1.0 6.55e+02  -3.0 1.81e-02 1.38e-03f  1
 748  1.6753754e+05 1.76e+02 3.86e+06  -1.0 6.57e+02  -2.6 1.07e-01 2.21e-02f  1
 749  1.6751971e+05 1.72e+02 3.83e+06  -1.0 6.17e+02  -3.1 1.69e-02 2.73e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 750  1.6752251e+05 1.63e+02 2.53e+06  -1.0 6.21e+02  -2.6 2.27e-01 5.22e-02h  1
 751  1.6751619e+05 1.62e+02 2.54e+06  -1.0 5.54e+02  -3.1 1.22e-03 6.04e-03f  1
 752  1.6751965e+05 1.57e+02

...

1862  1.9678450e+02 2.55e-01 2.64e+03  -5.7 2.09e+03  -5.8 5.92e-04 9.96e-09f 17
1863  1.9656831e+02 2.57e-01 2.57e+03  -5.7 4.14e+04  -6.2 4.13e-08 2.79e-04f  1
1864  1.9589768e+02 2.57e-01 2.57e+03  -5.7 8.72e+03  -5.8 9.72e-06 9.93e-05f  1
1865  1.9589747e+02 2.57e-01 2.53e+03  -5.7 3.76e+00  -1.8 1.66e-01 4.09e-05h  1
1866  1.9588792e+02 2.57e-01 2.53e+03  -5.7 2.47e+02  -2.3 8.10e-04 5.91e-04f  1
1867  1.9574188e+02 2.56e-01 2.52e+03  -5.7 6.10e+01  -2.7 6.91e-03 3.31e-03f  1
1868  1.9574188e+02 2.56e-01 2.52e+03  -5.7 1.01e+02  -3.2 2.09e-07 1.75e-08h  2
1869  1.9574188e+02 2.56e-01 2.52e+03  -5.7 3.25e+03  -3.7 2.00e-04 1.26e-09h  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1870  1.9507300e+02 2.58e-01 2.51e+03  -5.7 6.08e+02  -4.2 1.92e-08 1.65e-03f  1
1871  1.9502789e+02 2.58e-01 2.71e+03  -5.7 3.74e+02  -4.6 7.36e-03 4.32e-04f  1
1872  1.9450201e+02 2.58e-01 2.72e+03  -5.7 3.42e+03  -5.1 2.92e-05 2.48e-04f  1
1873  1.9439623e+02 2.58e-01

1977  1.2939305e+02 5.05e+00 5.06e+03  -5.7 3.18e-01   1.0 1.00e-03 2.20e-03h  1
1978  1.2939305e+02 5.05e+00 5.24e+03  -5.7 6.74e-01   1.4 3.18e-02 5.17e-05f  2
1979  1.2939308e+02 5.05e+00 1.07e+04  -5.7 1.32e-01   1.8 8.88e-01 1.86e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1980  1.2939356e+02 4.79e+00 8.33e+03  -5.7 2.50e-01   1.3 1.45e-01 5.22e-02h  1
1981  1.2939366e+02 4.79e+00 6.00e+04  -5.7 3.07e-01   0.9 1.00e+00 6.28e-05h  1
1982  1.2958730e+02 4.25e+00 2.79e+04  -5.7 5.01e-01   0.4 1.00e+00 1.13e-01h  1
1983  1.3111606e+02 5.76e-02 9.22e+00  -5.7 4.54e-01  -0.1 1.00e+00 1.00e+00h  1
1984  1.3113932e+02 6.73e-06 1.81e-01  -5.7 1.50e-02  -0.6 1.00e+00 1.00e+00h  1
1985  1.3113395e+02 8.79e-08 3.62e-01  -5.7 4.14e-02  -1.0 1.00e+00 9.94e-01f  1
1986  1.3113109e+02 7.14e-08 4.15e+00  -5.7 1.23e-01  -1.5 1.00e+00 1.87e-01f  1
1987  1.3108755e+02 2.92e-05 3.51e-03  -5.7 3.49e-01  -2.0 1.00e+00 1.00e+00f  1
1988  1.3104562e+02 4.94e-05

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2100  8.2952108e+01 1.44e+00 2.72e+02  -5.7 6.18e-02   0.7 1.00e+00 9.13e-04h  1
2101  8.2858086e+01 1.12e-03 2.89e+02  -5.7 8.15e-02   0.2 1.00e+00 5.29e-01h  1
2102  8.2821332e+01 1.05e-03 3.54e+02  -5.7 2.40e-01  -0.3 4.86e-01 9.12e-02h  1
2103  8.2427475e+01 4.71e-03 2.15e+02  -5.7 6.85e-01  -0.7 2.75e-01 3.45e-01h  1
2104  8.2114713e+01 6.25e-03 2.20e+02  -5.7 1.69e+00  -1.2 1.48e-01 9.91e-02h  1
2105  8.1917770e+01 5.04e-03 3.66e+02  -5.7 6.34e-01  -0.8 4.38e-01 1.70e-01h  1
2106  8.1871004e+01 1.36e-03 8.52e+01  -5.7 2.72e-02   0.5 1.00e+00 7.35e-01h  1
2107  8.1706280e+01 5.58e-04 6.88e-01  -5.7 8.83e-02   0.1 1.00e+00 1.00e+00h  1
2108  8.1645524e+01 7.90e-05 9.99e-02  -5.7 3.24e-02   0.5 1.00e+00 1.00e+00h  1
2109  8.1462999e+01 6.87e-04 1.32e+00  -5.7 9.73e-02   0.0 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2110  8.1399774e+01 1.39e-04

2216  7.3905619e+01 1.46e-11 2.19e-04  -5.7 1.34e-03  -0.8 1.00e+00 1.00e+00h  1
2217  7.3905616e+01 5.82e-11 1.64e-04  -5.7 3.02e-03  -1.3 1.00e+00 1.00e+00h  1
2218  7.3905608e+01 2.33e-10 1.25e-04  -5.7 6.89e-03  -1.7 1.00e+00 1.00e+00h  1
2219  7.3905581e+01 1.04e-09 9.90e-05  -5.7 1.64e-02  -2.2 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2220  7.3905499e+01 5.01e-09 8.63e-05  -5.7 4.30e-02  -2.7 1.00e+00 1.00e+00h  1
2221  7.3905249e+01 2.63e-08 8.23e-05  -5.7 1.23e-01  -3.2 1.00e+00 1.00e+00h  1
2222  7.3904501e+01 2.16e-07 8.17e-05  -5.7 3.66e-01  -3.7 1.00e+00 1.00e+00h  1
2223  7.3902263e+01 3.80e-05 1.71e-04  -5.7 1.09e+00  -4.1 1.00e+00 1.00e+00h  1
2224  7.3901920e+01 2.86e-05 2.73e+01  -5.7 3.17e+00  -4.6 1.00e+00 5.30e-02h  1
2225  7.3900017e+01 2.16e-04 6.14e+01  -5.7 7.71e+00  -5.1 1.00e+00 1.20e-01f  1
2226  7.3858930e+01 1.06e-01 1.14e+01  -5.7 2.12e+01  -5.6 1.00e+00 9.41e-01f  1
2227  7.3822609e+01 1.56e-01

2328r 7.3405085e+01 5.23e-01 1.00e+03  -0.3 0.00e+00   1.3 0.00e+00 4.34e-07R  3
2329r 7.3405077e+01 5.24e-01 1.00e+03  -0.3 6.25e+09    -  3.99e-09 1.73e-10f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2330r 7.3621803e+01 5.47e+00 1.01e+03  -0.3 9.15e+05    -  1.01e-04 3.58e-04f  1
2331r 7.3994339e+01 5.59e+00 9.99e+02  -0.3 4.74e+04    -  1.95e-03 1.09e-03f  1
2332r 7.4576728e+01 8.52e+00 9.98e+02  -0.3 2.74e+04    -  1.95e-02 3.58e-03f  1
2333r 7.6719476e+01 5.47e+01 9.83e+02  -0.3 7.85e+03    -  1.05e-02 2.85e-02f  1
2334r 8.0980541e+01 5.82e+01 9.24e+02  -0.3 4.31e+03    -  6.20e-02 4.87e-02f  1
2335r 9.3521082e+01 9.34e+01 7.41e+02  -0.3 3.05e+03    -  2.08e-01 1.82e-01f  1
2336r 1.0531359e+02 1.23e+02 7.34e+02  -0.3 3.24e+00   0.0 2.76e-01 8.09e-01f  1
2337r 1.0615536e+02 2.95e+01 2.62e+02  -0.3 2.91e-01   1.3 5.85e-01 7.96e-01f  1
2338r 1.0838445e+02 3.31e+00 2.44e+00  -0.3 1.99e-01   0.9 1.00e+00 1.00e+00f  1
2339r 1.1175348e+02 2.31e+01

In [5]:
solve_pyomo(para_truth[1])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 187r 1.6804111e+04 9.59e+04 2.01e+04  -4.5 4.77e+01  -0.1 1.62e-01 5.94e-03f  1
 188r 1.6866038e+04 9.59e+04 1.92e+04  -4.5 1.41e+02  -0.6 4.98e-02 1.44e-01f  1
 189r 1.6874902e+04 9.59e+04 1.61e+04  -4.5 5.34e+01  -0.2 1.57e-01 5.47e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.6979959e+04 9.59e+04 1.55e+04  -4.5 1.58e+02  -0.7 4.32e-02 2.18e-01f  1
 191r 1.6979968e+04 9.59e+04 8.74e+03  -4.5 5.97e+01  -0.3 4.25e-01 4.71e-05f  1
 192r 1.7016912e+04 9.59e+04 8.75e+03  -4.5 1.77e+02  -0.7 3.48e-03 7.07e-02f  1
 193r 1.7016917e+04 9.59e+04 4.23e+03  -4.5 6.70e+01  -0.3 5.66e-01 2.90e-05f  1
 194r 1.7049606e+04 9.58e+04 4.00e+03  -4.5 1.98e+02  -0.8 6.02e-02 5.58e-02f  1
 195r 1.7049651e+04 9.58e+04 4.00e+03  -4.5 7.52e+01  -0.4 5.47e-01 2.06e-04f  1
 196r 1.7439607e+04 9.57e+04 1.56e+03  -4.5 2.22e+02  -0.8 1.00e-01 6.01e-01f  1
 197r 1.7439629e+04 9.57e+04 1.56e+03  -4.5 8.32e+01  -0.4 2.75e-01 8.97e-05f  1
 198r 1.7809293e+04 9.56e+04

 502  2.2056511e+05 1.98e+03 1.44e+06  -1.0 2.65e+03  -0.6 1.12e-02 8.67e-04h  1
 503  2.2057827e+05 1.98e+03 1.44e+06  -1.0 2.64e+03  -1.0 3.09e-03 2.29e-03h  1
 504  2.2058393e+05 1.97e+03 1.84e+06  -1.0 2.61e+03  -0.6 4.29e-02 9.22e-04h  1
 505  2.2059900e+05 1.97e+03 1.84e+06  -1.0 2.56e+03  -1.1 2.28e-03 2.91e-03h  1
 506  2.2060236e+05 1.97e+03 1.92e+06  -1.0 2.57e+03  -0.7 9.96e-03 5.99e-04h  1
 507  2.2061779e+05 1.96e+03 1.92e+06  -1.0 2.55e+03  -1.1 6.35e-03 3.04e-03h  1
 508  2.2062369e+05 1.96e+03 2.24e+06  -1.0 2.55e+03  -0.7 4.06e-02 1.09e-03h  1
 509  2.2064915e+05 1.95e+03 2.22e+06  -1.0 2.53e+03  -1.2 3.85e-03 5.37e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 510  2.2065145e+05 1.95e+03 2.31e+06  -1.0 2.52e+03  -0.8 1.28e-02 4.55e-04h  1
 511  2.2065922e+05 1.94e+03 2.31e+06  -1.0 2.49e+03  -1.2 2.39e-03 1.88e-03h  1
 512  2.2066082e+05 1.94e+03 2.43e+06  -1.0 2.46e+03  -0.8 1.69e-02 3.57e-04h  1
 513  2.2067131e+05 1.94e+03

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 610  2.2315055e+05 4.89e+01 1.82e+05  -1.0 6.31e+02  -1.5 2.26e-02 5.07e-03h  1
 611  2.2315228e+05 4.88e+01 1.94e+05  -1.0 7.38e+02  -1.9 1.67e-02 3.58e-03h  1
 612  2.2278881e+05 4.80e+01 1.93e+05  -1.0 1.07e+04    -  3.09e-03 1.57e-02f  1
 613  2.2233446e+05 4.71e+01 1.88e+05  -1.0 1.05e+04    -  2.96e-02 2.02e-02f  1
 614  2.2172100e+05 4.60e+01 1.70e+05  -1.0 1.12e+04    -  8.79e-02 2.48e-02f  1
 615  2.2044463e+05 4.44e+01 1.71e+05  -1.0 1.31e+04    -  1.24e-03 4.32e-02f  1
 616  2.2042247e+05 4.44e+01 1.56e+05  -1.0 1.32e+04    -  8.07e-02 7.43e-04f  1
 617  2.2042275e+05 4.42e+01 1.54e+05  -1.0 1.41e+03  -2.4 3.55e-03 4.81e-03h  1
 618  2.2041968e+05 4.37e+01 1.51e+05  -1.0 1.06e+03  -2.9 5.78e-04 1.09e-02f  1
 619  2.2041968e+05 4.37e+01 1.53e+05  -1.0 9.34e+02  -2.5 3.81e-03 1.79e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 620  2.2038072e+05 4.18e+01

 729  2.1245955e+05 7.14e+00 6.54e+05  -1.0 1.16e+01   1.0 2.25e-02 1.36e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 730  2.1245963e+05 7.12e+00 3.83e+05  -1.0 1.33e+01   0.6 2.32e-01 1.67e-03h  1
 731  2.1246007e+05 7.11e+00 7.27e+05  -1.0 6.68e+01   1.0 2.55e-02 2.30e-03h  1
 732  2.1246140e+05 7.07e+00 9.27e+05  -1.0 7.88e+01   0.5 1.83e-02 6.04e-03h  1
 733  2.1246920e+05 6.84e+00 3.33e+06  -1.0 8.90e+01   0.0 5.71e-01 3.19e-02h  1
 734  2.1248598e+05 6.36e+00 2.34e+06  -1.0 8.73e+01  -0.5 3.36e-01 7.06e-02h  1
 735  2.1248847e+05 6.29e+00 2.29e+06  -1.0 8.63e+01  -0.9 1.84e-02 9.99e-03h  1
 736  2.1248887e+05 6.28e+00 2.29e+06  -1.0 7.14e+01  -0.5 4.53e-04 2.25e-03h  1
 737  2.1249072e+05 6.22e+00 1.48e+05  -1.0 7.39e+01  -1.0 6.90e-01 9.40e-03h  1
 738  2.1249625e+05 6.02e+00 1.26e+05  -1.0 6.42e+01  -1.5 2.06e-02 3.22e-02h  1
 739  2.1248763e+05 5.39e+00 1.04e+05  -1.0 9.53e+01  -1.9 3.51e-02 1.05e-01f  1
iter    objective    inf_pr 

...

1679  2.7596123e+03 1.07e-01 2.16e+03  -3.8 1.96e+02  -2.8 2.29e-01 3.68e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1680  2.5475432e+03 3.03e+00 1.46e+03  -3.8 7.46e+02  -3.3 1.96e-02 7.13e-02f  1
1681  2.4914292e+03 3.04e+00 9.29e+02  -3.8 2.29e+02  -2.9 1.12e-01 5.81e-02f  1
1682  2.4471671e+03 3.30e+00 1.40e+03  -3.8 1.51e+03  -3.3 8.65e-06 7.24e-03f  1
1683  2.4470304e+03 3.30e+00 2.75e+03  -3.8 2.85e+02  -2.9 1.33e-01 1.13e-04f  1
1684  2.3706240e+03 4.11e+00 3.05e+03  -3.8 2.35e+03  -3.4 3.61e-06 8.07e-03f  1
1685  2.3705261e+03 4.11e+00 4.89e+03  -3.8 3.65e+02  -3.0 9.60e-02 6.33e-05f  1
1686  2.2872249e+03 5.28e+00 4.51e+03  -3.8 2.27e+04  -3.4 2.73e-07 9.11e-04f  1
1687  2.2871567e+03 5.28e+00 6.58e+03  -3.8 5.14e+02  -3.0 5.79e-02 3.11e-05f  1
1688  2.1137801e+03 6.46e+00 4.44e+03  -3.8 1.45e+02  -2.6 1.12e-03 2.77e-01f  1
1689  2.1119524e+03 6.46e+00 3.08e+03  -3.8 5.29e+04  -3.1 1.96e-04 7.88e-06f  1
iter    objective    inf_pr 

1797  1.8457435e+03 2.30e-01 2.13e+04  -3.8 5.32e+01  -2.2 7.96e-02 3.83e-02f  1
1798  1.8285461e+03 2.19e-01 2.01e+04  -3.8 1.55e+02  -2.7 3.55e-02 4.74e-02f  1
1799  1.8282308e+03 2.19e-01 2.03e+04  -3.8 4.13e+02  -3.2 1.10e-02 4.52e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1800  1.7793081e+03 2.14e-01 1.97e+04  -3.8 1.04e+03  -3.6 1.03e-02 2.79e-02f  1
1801  1.7205433e+03 2.11e-01 1.98e+04  -3.8 1.99e+03  -4.1 5.96e-03 1.77e-02f  1
1802  1.7072547e+03 2.11e-01 1.98e+04  -3.8 2.72e+03  -4.6 8.78e-06 1.36e-03f  1
1803  1.7071137e+03 2.11e-01 1.63e+04  -3.8 3.30e+03  -5.1 4.12e-02 2.58e-05f  1
1804  1.6406522e+03 2.42e-01 1.71e+04  -3.8 3.51e+03  -5.6 6.72e-06 1.04e-02f  1
1805  1.6405009e+03 2.42e-01 1.54e+04  -3.8 3.43e+03  -6.0 1.86e-02 2.74e-05f  1
1806  1.4093354e+03 2.21e+00 1.83e+04  -3.8 3.70e+03  -6.5 4.76e-03 4.10e-02f  1
1807  1.3769049e+03 2.35e+00 1.73e+04  -3.8 2.26e+04  -7.0 1.20e-02 6.19e-03f  1
1808  1.3171977e+03 2.49e+00

1915  4.4412303e+02 3.14e+00 4.77e+02  -3.8 1.60e+03  -6.6 2.02e-05 8.16e-03f  1
1916  4.3768761e+02 3.19e+00 2.54e+02  -3.8 1.52e+04  -7.1 5.93e-03 9.74e-03f  1
1917  4.3336559e+02 3.18e+00 4.57e+03  -3.8 3.06e+03  -6.7 9.16e-02 7.35e-03f  1
1918  4.3159560e+02 3.18e+00 4.33e+03  -3.8 4.77e+04  -7.2 1.70e-06 7.96e-04f  1
1919  4.3156470e+02 3.17e+00 3.45e+03  -3.8 1.69e+00   0.7 8.70e-04 4.23e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1920  4.3156450e+02 3.17e+00 3.46e+03  -3.8 2.87e-01   2.9 5.71e-03 4.12e-04f  2
1921  4.3156412e+02 3.17e+00 3.94e+03  -3.8 2.10e-01   2.4 7.97e-03 7.04e-04h  1
1922  4.3156504e+02 3.16e+00 1.35e+04  -3.8 5.50e-01   1.9 7.46e-03 3.63e-04h  1
1923  4.3160683e+02 3.14e+00 1.18e+04  -3.8 1.45e+00   1.5 5.87e-03 6.98e-03h  1
1924  4.3162504e+02 3.13e+00 4.19e+04  -3.8 1.87e+00   1.0 4.96e-02 2.67e-03h  2
1925  4.3191352e+02 3.01e+00 2.22e+05  -3.8 1.98e+00   0.5 1.00e+00 4.02e-02h  1
1926  4.3547923e+02 1.48e+00

2035  1.7023575e+02 3.53e+00 1.87e+04  -3.8 1.07e+03    -  3.55e-02 2.24e-03h  1
2036  1.6962039e+02 3.52e+00 1.88e+04  -3.8 5.79e+03    -  7.38e-05 4.06e-02f  1
2037  1.6942383e+02 3.53e+00 1.86e+04  -3.8 5.08e+04    -  5.63e-03 1.26e-03f  1
2038  1.6918845e+02 3.50e+00 1.76e+04  -3.8 7.27e+03    -  4.65e-02 1.77e-02f  1
2039  1.6900897e+02 3.34e+00 1.45e+04  -3.8 5.01e+02    -  1.66e-01 4.63e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2040  1.6891491e+02 3.03e+00 1.41e+04  -3.8 1.78e+02    -  3.29e-02 9.49e-02h  1
2041  1.6873193e+02 2.88e+00 1.41e+04  -3.8 2.13e+03    -  8.60e-03 1.39e-01h  1
2042  1.6864526e+02 2.24e+00 1.16e+04  -3.8 2.71e+02    -  1.75e-01 2.28e-01h  1
2043  1.5745893e+02 2.30e+00 1.41e+04  -3.8 1.42e+03    -  2.78e-05 1.15e-01f  1
2044  1.5698197e+02 2.30e+00 1.36e+04  -3.8 2.94e+04    -  6.23e-03 1.19e-03f  1
2045  1.5701730e+02 2.18e+00 9.25e+04  -3.8 1.12e-01   1.6 1.00e+00 5.44e-02h  1
2046  1.5704487e+02 2.10e+00

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2170  7.4514850e+01 5.72e-08 5.48e-01  -3.8 3.56e-02  -0.9 1.00e+00 1.00e+00h  1
2171  7.4513942e+01 6.94e-01 5.86e+00  -3.8 1.05e-01  -1.4 1.00e+00 1.00e+00h  1
2172  7.4513412e+01 1.83e-07 1.58e+00  -3.8 4.67e-02  -0.9 1.00e+00 1.00e+00h  1
2173  7.4512530e+01 5.72e-01 2.25e+02  -3.8 6.15e-01  -1.4 1.00e+00 1.64e-01h  2
2174  7.4510737e+01 4.72e-01 1.20e+01  -3.8 1.11e-01  -1.0 1.00e+00 9.08e-01h  1
2175  7.4491946e+01 3.69e+00 3.90e+01  -3.8 8.88e+02    -  7.88e-02 8.26e-02h  1
2176  7.4481259e+01 3.12e+00 5.18e+01  -3.8 3.50e+02    -  2.94e-01 1.57e-01h  1
2177  7.4482720e+01 1.48e+00 2.67e+01  -3.8 3.57e-01  -1.5 7.41e-01 6.50e-01h  1
2178  7.4484992e+01 1.95e-01 3.62e+01  -3.8 3.74e-02  -0.1 1.00e+00 9.22e-01h  1
2179  7.4465563e+01 1.41e+01 7.12e+02  -3.8 1.29e+03    -  4.21e-02 1.03e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2180  7.4472562e+01 7.58e+00

In [6]:
solve_pyomo(para_truth[2])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 197r 1.7506334e+04 9.59e+04 1.19e+04  -6.7 3.37e+03  -1.9 8.24e-03 3.27e-07f  1
 198r 1.7756198e+04 9.58e+04 1.16e+04  -6.7 9.96e+03  -2.4 4.86e-03 7.29e-03f  1
 199r 1.7756294e+04 9.58e+04 1.16e+04  -6.7 4.93e+04  -2.9 1.45e-04 1.04e-06f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 200r 1.7756294e+04 9.58e+04 1.16e+04  -6.7 1.09e+04  -2.5 5.47e-07 1.22e-08f  1
 201r 1.7767756e+04 9.58e+04 1.14e+04  -6.7 4.26e+03  -2.0 2.64e-05 8.09e-04f  1
 202r 1.7767756e+04 9.58e+04 1.14e+04  -6.7 1.25e+04  -2.5 2.16e-04 2.23e-09f  1
 203r 1.8496385e+04 9.56e+04 5.42e+03  -6.7 4.76e+03  -2.1 9.07e-03 4.47e-02f  1
 204r 1.8505134e+04 9.56e+04 5.25e+03  -6.7 1.98e+03  -1.7 3.56e-09 1.61e-03f  1
 205r 1.8505153e+04 9.56e+04 5.12e+03  -6.7 6.47e+03  -2.1 3.56e-03 1.25e-06f  1
 206r 1.8577947e+04 9.56e+04 4.81e+03  -6.7 2.39e+03  -1.7 9.64e-04 1.25e-02f  1
 207r 1.8625023e+04 9.55e+04 4.59e+03  -6.7 7.30e+03  -2.2 7.05e-03 2.96e-03f  1
 208r 1.8625023e+04 9.55e+04

 509  1.7219967e+05 2.46e+03 7.02e+05  -1.0 2.74e+03  -1.9 7.92e-03 4.24e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 510  1.7220973e+05 2.46e+03 7.02e+05  -1.0 2.46e+03  -1.5 1.39e-03 1.14e-03h  1
 511  1.7222262e+05 2.45e+03 7.10e+05  -1.0 2.46e+03  -2.0 1.28e-02 1.54e-03h  1
 512  1.7224245e+05 2.45e+03 8.87e+05  -1.0 2.45e+03  -1.6 5.74e-02 2.25e-03h  1
 513  1.7227857e+05 2.44e+03 9.21e+05  -1.0 2.71e+03  -2.0 3.94e-02 4.43e-03h  1
 514  1.7231648e+05 2.43e+03 1.18e+06  -1.0 2.47e+03  -1.6 7.11e-02 4.38e-03h  1
 515  1.7238676e+05 2.41e+03 1.23e+06  -1.0 2.70e+03  -2.1 7.96e-02 8.46e-03h  1
 516  1.7240762e+05 2.40e+03 1.22e+06  -1.0 2.66e+03  -1.7 9.92e-04 2.47e-03h  1
 517  1.7242558e+05 2.39e+03 1.24e+06  -1.0 2.89e+03  -2.1 2.52e-02 2.17e-03h  1
 518  1.7250232e+05 2.37e+03 1.24e+06  -1.0 2.69e+03  -1.7 1.02e-02 9.11e-03h  1
 519  1.7251678e+05 2.37e+03 1.24e+06  -1.0 3.51e+03  -2.2 1.27e-02 1.83e-03h  1
iter    objective    inf_pr 

 628  1.7745518e+05 5.62e+02 3.34e+07  -1.0 1.23e+03  -0.9 2.30e-04 4.11e-03h  1
 629  1.7745521e+05 5.62e+02 3.31e+07  -1.0 1.17e+03  -1.4 1.20e-02 4.91e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 630  1.7745393e+05 5.62e+02 3.39e+07  -1.0 1.49e+03  -1.0 6.03e-03 1.58e-03f  1
 631  1.7742879e+05 5.50e+02 3.24e+07  -1.0 6.49e+03  -1.4 1.52e-04 2.04e-02f  1
 632  1.7742962e+05 5.50e+02 3.23e+07  -1.0 1.57e+03  -1.0 1.61e-04 6.88e-04h  1
 633  1.7742989e+05 5.50e+02 3.36e+07  -1.0 1.58e+03  -0.6 6.73e-03 2.13e-04h  1
 634  1.7746439e+05 5.36e+02 3.21e+07  -1.0 1.60e+03  -1.1 3.26e-04 2.44e-02h  1
 635  1.7756315e+05 5.05e+02 3.15e+07  -1.0 1.58e+03  -1.5 6.39e-04 5.80e-02h  1
 636  1.7770742e+05 4.51e+02 2.76e+07  -1.0 1.47e+03  -1.1 7.24e-02 1.06e-01h  1
 637  1.7770881e+05 4.51e+02 2.76e+07  -1.0 1.29e+03  -0.7 8.04e-04 1.20e-03h  1
 638  1.7770882e+05 4.51e+02 2.77e+07  -1.0 1.28e+03  -1.2 1.14e-02 1.47e-05h  1
 639  1.7770903e+05 4.51e+02

 746  1.7278356e+05 1.23e+01 2.24e+05  -1.0 2.58e+03  -3.5 3.40e-02 4.49e-02f  1
 747  1.7276780e+05 1.23e+01 1.30e+05  -1.0 5.81e+03  -3.9 1.17e-01 8.47e-04f  1
 748  1.7252676e+05 8.07e+00 7.25e+05  -1.0 2.24e+02  -2.6 1.94e-02 3.46e-01f  1
 749  1.7187267e+05 6.12e+00 7.33e+04  -1.0 7.68e+02  -3.1 4.51e-01 2.61e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 750  1.7152362e+05 7.96e+00 1.64e+05  -1.0 2.35e+03  -3.6 8.01e-02 4.53e-02f  1
 751  1.7128138e+05 1.65e+01 1.69e+05  -1.0 1.00e+04  -4.0 9.62e-03 7.45e-03f  1
 752  1.7118822e+05 1.65e+01 1.89e+05  -1.0 1.82e+04  -4.5 8.05e-03 1.56e-03f  1
 753  1.7046165e+05 1.66e+01 1.93e+05  -1.0 7.30e+04  -5.0 4.64e-03 3.10e-03f  1
 754  1.7044706e+05 1.66e+01 2.91e+05  -1.0 1.97e+04  -4.6 3.18e-02 2.21e-04f  1
 755  1.6809707e+05 1.76e+01 2.80e+05  -1.0 7.79e+04  -5.0 6.36e-03 9.23e-03f  1
 756  1.6714861e+05 1.76e+01 3.14e+05  -1.0 2.15e+04  -4.6 2.53e-02 1.30e-02f  1
 757  1.6384319e+05 1.95e+01

...

1103  7.5880483e+03 5.12e+01 2.30e+04  -3.8 9.51e+03  -4.7 9.34e-04 3.12e-04f  1
1104  7.5878672e+03 5.12e+01 2.44e+04  -3.8 8.08e+03    -  1.45e-02 6.33e-05f  1
1105  7.5754758e+03 5.10e+01 2.40e+04  -3.8 3.24e+03    -  2.10e-06 2.82e-03f  1
1106  7.5642792e+03 5.10e+01 2.41e+04  -3.8 3.49e+06  -5.1 2.00e-09 1.03e-06f  1
1107  7.5313644e+03 5.08e+01 2.39e+04  -3.8 2.48e+03  -3.8 6.15e-03 4.11e-03f  1
1108  7.4667878e+03 5.07e+01 2.40e+04  -3.8 1.19e+04  -4.3 1.17e-05 1.59e-03f  1
1109  7.3195346e+03 4.99e+01 2.38e+04  -3.8 2.82e+03  -3.9 1.35e-02 1.57e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1110  7.0587790e+03 4.96e+01 2.39e+04  -3.8 1.37e+04  -4.3 3.71e-03 5.50e-03f  1
1111  4.6793734e+03 6.64e+01 2.78e+04  -3.8 3.14e+03  -3.9 9.15e-03 2.24e-01f  1
1112  4.0073363e+03 7.63e+01 2.75e+04  -3.8 1.15e+04  -4.4 8.41e-03 1.59e-02f  1
1113  3.6347427e+03 7.88e+01 2.73e+04  -3.8 4.06e+03  -4.0 9.59e-03 2.54e-02f  1
1114  3.6346822e+03 7.88e+01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1240  1.9310446e+02 7.74e+00 3.62e+03  -3.8 1.87e+03    -  1.74e-02 4.53e-02f  1
1241  1.9099058e+02 7.73e+00 6.77e+03  -3.8 1.42e+03    -  2.08e-01 5.33e-03f  1
1242  1.6826205e+02 1.08e+01 5.06e+03  -3.8 1.48e+03    -  6.54e-05 8.01e-02f  1
1243  1.6781368e+02 1.02e+01 4.71e+03  -3.8 1.59e+00  -0.7 4.80e-02 5.71e-02h  1
1244  1.6682735e+02 9.70e+00 4.48e+03  -3.8 3.98e+00  -1.1 5.14e-02 5.07e-02f  1
1245  1.6424352e+02 9.43e+00 3.96e+03  -3.8 1.24e+01  -1.6 5.21e-02 6.41e-02f  1
1246  1.6037927e+02 9.44e+00 2.62e+03  -3.8 3.58e+01  -2.1 2.01e-02 5.47e-02f  1
1247  1.6037381e+02 9.44e+00 2.93e+03  -3.8 2.26e+00  -0.8 3.31e-02 5.00e-04h  1
1248  1.6025615e+02 9.40e+00 3.11e+03  -3.8 3.19e+01  -1.2 9.69e-03 4.34e-03f  1
1249  1.5978612e+02 9.32e+00 3.36e+03  -3.8 2.99e+01  -0.8 1.76e-02 3.35e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1250  1.5960648e+02 9.26e+00

1367  8.6617989e+01 9.23e+00 4.26e+04  -5.7 1.44e+00   0.3 7.44e-01 2.67e-02h  5
1368  8.6686876e+01 9.18e+00 3.77e+04  -5.7 2.33e+00  -0.2 1.59e-01 1.73e-02h  5
1369  8.6910382e+01 9.05e+00 3.79e+04  -5.7 1.16e+00   0.3 7.87e-01 5.42e-02h  4
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1370  8.7118534e+01 8.86e+00 8.01e+04  -5.7 1.06e+00   0.7 8.98e-01 5.02e-02h  4
1371  8.8531066e+01 7.88e+00 3.90e+04  -5.7 1.09e+00   0.2 1.00e+00 2.12e-01h  1
1372  8.8736189e+01 7.80e+00 5.87e+04  -5.7 1.07e+00   0.6 3.86e-01 4.34e-02h  2
1373  8.8740516e+01 7.80e+00 8.92e+04  -5.7 7.23e+00   0.2 1.94e-01 1.43e-03h  6
1374  8.9165229e+01 7.27e+00 4.63e+03  -5.7 1.03e+00  -0.3 1.00e+00 6.86e-02h  1
1375  8.9438185e+01 7.28e+00 8.27e+04  -5.7 7.75e-01   1.0 9.43e-01 1.13e-01h  1
1376  8.9769506e+01 6.81e+00 3.28e+04  -5.7 7.60e-01   0.5 1.00e+00 8.86e-02h  1
1377  9.0695883e+01 5.57e+00 7.22e+03  -5.7 7.70e-01   0.1 1.00e+00 1.93e-01h  1
1378  9.0960482e+01 5.20e+00

1483  8.5498979e+01 3.03e-02 3.55e-01  -5.7 1.17e+01  -4.6 1.00e+00 1.00e+00f  1
1484  8.5121725e+01 8.67e-02 1.76e+01  -5.7 3.36e+01  -5.1 5.62e-01 5.44e-01f  1
1485  8.5121607e+01 8.67e-02 1.76e+01  -5.7 9.19e+01  -5.5 6.49e-05 6.34e-05h  1
1486  8.5121597e+01 8.67e-02 1.76e+01  -5.7 2.26e+02  -6.0 2.39e-06 2.25e-06f  1
1487  8.5121595e+01 8.67e-02 2.66e+03  -5.7 2.53e+02  -6.5 7.06e-01 3.93e-07f  2
1488  8.5109032e+01 8.57e-02 2.39e+03  -5.7 8.83e+02  -7.0 1.02e-01 2.90e-02h  1
1489  8.5089260e+01 8.59e-02 2.39e+03  -5.7 1.41e+05  -7.5 1.12e-08 3.83e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1490  8.4877460e+01 8.52e-02 2.28e+03  -5.7 3.05e+02  -6.1 4.68e-02 4.81e-02f  1
1491  8.4606969e+01 1.04e-01 2.23e+03  -5.7 8.39e+02  -6.6 1.97e-02 2.26e-02f  1
1492  8.4268689e+01 1.78e-01 2.23e+03  -5.7 2.11e+03  -7.1 6.29e-03 1.83e-02f  1
1493  8.3990635e+01 2.39e-01 2.09e+03  -5.7 1.35e+03  -7.6 5.21e-02 2.15e-02f  1
1494  8.3757708e+01 2.98e-01

1613  7.6654083e+01 9.99e+00 4.87e+02  -5.7 1.01e-01  -1.7 1.00e+00 1.39e-02h  1
1614  7.6650040e+01 1.91e+00 1.38e+03  -5.7 1.01e+02    -  3.68e-02 8.09e-01h  1
1615  7.6650683e+01 2.14e-07 8.24e+02  -5.7 1.04e-01  -1.2 3.52e-01 1.00e+00h  1
1616  7.6650671e+01 1.72e-10 3.12e-01  -5.7 5.98e-03  -0.8 1.00e+00 1.00e+00h  1
1617  7.6650629e+01 9.60e-10 5.60e+00  -5.7 2.78e-02  -1.3 1.00e+00 1.00e+00h  1
1618  7.6650597e+01 1.03e-10 7.78e-01  -5.7 1.06e-02  -0.9 1.00e+00 1.00e+00h  1
1619  7.6650538e+01 4.99e-03 8.25e+01  -5.7 4.95e-02  -1.3 1.00e+00 5.97e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1620  7.6650504e+01 7.04e-03 4.84e+02  -5.7 7.56e-01  -1.8 1.75e-01 1.71e-02h  1
1621  7.6650323e+01 6.58e-03 4.53e+02  -5.7 1.33e+01    -  4.17e-02 1.41e-01h  1
1622  7.6647327e+01 2.00e-01 5.94e+02  -5.7 8.49e+03    -  5.60e-03 3.77e-03f  1
1623  7.6629543e+01 1.29e+01 2.26e+03  -5.7 1.03e+03    -  4.89e-02 1.91e-01h  1
1624  7.6624351e+01 1.26e+01

In [7]:
solve_pyomo(para_truth[3])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 187r 8.7756065e+03 9.78e+04 3.70e+03  -3.0 6.48e+02    -  6.45e-02 4.43e-02f  1
 188r 8.8355919e+03 9.77e+04 3.80e+03  -3.0 6.20e+02    -  2.65e-02 1.93e-02f  1
 189r 8.8356496e+03 9.77e+04 1.07e+02  -3.0 3.14e+00   0.4 1.00e+00 4.19e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 9.0448794e+03 9.77e+04 1.07e+02  -3.0 6.08e+02    -  6.12e-02 6.88e-02f  1
 191r 9.0466728e+03 9.77e+04 2.84e+02  -3.0 8.64e+00  -0.1 1.00e+00 4.80e-02f  1
 192r 9.1664936e+03 9.77e+04 3.15e+02  -3.0 5.66e+02    -  1.92e-02 4.23e-02f  1
 193r 9.2038364e+03 9.77e+04 3.76e+02  -3.0 2.39e+01  -0.6 2.12e-01 3.65e-01f  1
 194r 9.2429397e+03 9.77e+04 6.20e+00  -3.0 9.09e+00  -0.2 1.00e+00 1.00e+00f  1
 195r 9.2985973e+03 9.76e+04 1.91e+01  -3.0 2.55e+01  -0.6 5.27e-01 5.06e-01f  1
 196r 9.3092246e+03 9.76e+04 3.07e+01  -3.0 6.64e+01  -1.1 1.66e-02 3.60e-02f  1
 197r 9.3100066e+03 9.76e+04 1.24e+02  -3.0 3.66e+00   0.2 1.00e+00 5.01e-02f  1
 198r 9.3970331e+03 9.76e+04

 370r 3.2234402e+04 9.21e+04 8.30e+03  -4.5 7.35e+00   0.5 2.55e-01 1.04e-01f  1
 371r 3.2236875e+04 9.21e+04 3.94e+03  -4.5 2.74e+00   0.9 6.43e-01 2.88e-01f  1
 372r 3.2236875e+04 9.21e+04 2.85e+03  -4.5 8.15e+00   0.4 5.70e-01 7.04e-06f  1
 373r 3.2247138e+04 9.21e+04 3.09e+03  -4.5 2.41e+01  -0.0 3.28e-02 1.27e-01f  1
 374r 3.2259197e+04 9.21e+04 3.09e+03  -4.5 7.18e+01  -0.5 1.73e-02 5.02e-02f  1
 375r 3.2259197e+04 9.21e+04 5.88e+03  -4.5 4.38e-01   1.7 3.27e-01 2.74e-04f  1
 376r 3.2259774e+04 9.21e+04 1.98e+03  -4.5 1.29e+00   1.2 8.42e-01 1.39e-01f  1
 377r 3.2263326e+04 9.21e+04 1.38e+03  -4.5 3.83e+00   0.8 4.93e-01 2.79e-01f  1
 378r 3.2265425e+04 9.21e+04 7.73e+02  -4.5 1.44e+00   1.2 1.00e+00 4.43e-01f  1
 379r 3.2271211e+04 9.21e+04 6.40e+02  -4.5 4.29e+00   0.7 6.20e-01 4.01e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 380r 3.2272279e+04 9.21e+04 2.28e+03  -4.5 1.28e+01   0.2 5.50e-01 2.44e-02f  1
 381r 3.2332428e+04 9.21e+04

 507r 4.0832964e+04 8.99e+04 9.99e+02   5.0 0.00e+00   3.0 0.00e+00 1.88e-08R  2
 508r 4.0825066e+04 8.99e+04 9.99e+02   5.0 4.45e+08    -  7.15e-09 8.36e-09f  1
 509r 4.0775263e+04 8.99e+04 9.99e+02   2.9 1.03e+06    -  1.26e-05 2.52e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 510r 4.0837787e+04 8.99e+04 1.92e+03   2.9 3.63e+05    -  1.66e-03 2.04e-04f  1
 511r 4.2214703e+04 8.95e+04 2.15e+03   2.9 2.69e+05    -  1.75e-03 1.34e-03f  1
 512r 4.3565688e+04 8.92e+04 7.68e+03   2.9 3.01e+05    -  8.52e-03 1.17e-03f  1
 513r 4.5629375e+04 8.86e+04 8.28e+03   2.9 4.22e+05    -  1.90e-03 1.25e-03f  1
 514r 4.6602855e+04 8.84e+04 8.50e+03   2.9 4.05e+05    -  7.82e-04 6.13e-04f  1
 515r 4.7669414e+04 8.81e+04 1.02e+04   2.9 4.07e+05    -  1.76e-03 6.68e-04f  1
 516r 4.8205997e+04 8.80e+04 1.15e+04   2.9 2.62e+05    -  1.17e-03 5.50e-04f  1
 517r 4.9026846e+04 8.77e+04 1.54e+04   2.9 2.03e+05    -  2.94e-03 1.21e-03f  1
 518r 5.1115383e+04 8.71e+04

 762  2.8665326e+05 2.09e+02 5.84e+03  -1.0 1.59e+02  -0.2 9.91e-03 3.25e-03h  1
 763  2.8666487e+05 2.09e+02 6.23e+03  -1.0 1.53e+02   0.2 3.39e-02 1.88e-03h  1
 764  2.8669072e+05 2.08e+02 6.54e+03  -1.0 1.57e+02   0.6 1.41e-02 4.19e-03h  1
 765  2.8670716e+05 2.07e+02 1.02e+04  -1.0 1.73e+02   1.1 3.18e-02 2.68e-03h  1
 766  2.8672548e+05 2.07e+02 1.09e+04  -1.0 1.77e+02   0.6 1.39e-02 3.06e-03h  1
 767  2.8674209e+05 2.06e+02 1.67e+04  -1.0 1.82e+02   1.0 2.56e-02 2.77e-03h  1
 768  2.8676826e+05 2.05e+02 1.68e+04  -1.0 1.97e+02   0.5 6.24e-03 4.54e-03h  1
 769  2.8677014e+05 2.05e+02 2.73e+04  -1.0 1.89e+02   0.9 3.49e-02 3.26e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 770  2.8678141e+05 2.05e+02 3.08e+04  -1.0 1.99e+02   0.5 3.41e-02 2.05e-03h  1
 771  2.8679482e+05 2.04e+02 3.39e+04  -1.0 2.30e+02  -0.0 2.60e-02 2.74e-03h  1
 772  2.8683676e+05 2.02e+02 3.53e+04  -1.0 2.71e+02  -0.5 4.36e-02 9.48e-03h  1
 773  2.8683783e+05 2.02e+02

...

1955  2.6533951e+04 2.57e-01 4.57e+03  -3.8 7.36e+03  -3.8 3.83e-03 7.89e-06f  1
1956  2.6492689e+04 1.04e+00 4.86e+03  -3.8 2.45e+04  -4.3 5.81e-03 7.92e-04f  1
1957  2.6491920e+04 1.04e+00 5.14e+03  -3.8 8.06e+03  -3.8 4.45e-03 3.91e-05f  1
1958  2.6353110e+04 1.05e+00 5.07e+03  -3.8 2.34e+04  -4.3 1.66e-03 2.56e-03f  1
1959  2.6283122e+04 1.05e+00 5.35e+03  -3.8 9.02e+03  -3.9 7.70e-03 3.22e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1960  2.6274000e+04 1.05e+00 5.34e+03  -3.8 2.63e+04  -4.4 3.99e-06 1.44e-04f  1
1961  2.6197848e+04 1.05e+00 6.52e+03  -3.8 1.02e+04  -3.9 2.13e-02 2.74e-03f  1
1962  2.6169157e+04 1.05e+00 6.63e+03  -3.8 2.77e+04  -4.4 1.78e-03 4.33e-04f  1
1963  2.5963812e+04 1.04e+00 8.08e+03  -3.8 1.12e+04  -4.0 2.76e-02 6.97e-03f  1
1964  2.5961923e+04 1.04e+00 8.89e+03  -3.8 2.62e+04  -4.5 7.47e-03 2.82e-05f  1
1965  2.5935180e+04 1.04e+00 1.01e+04  -3.8 9.43e+03  -4.0 1.36e-02 1.13e-03f  1
1966  2.5820121e+04 1.19e+00

2073  3.8528410e+03 3.17e+01 3.20e+04  -3.8 7.10e+02  -3.2 6.75e-05 3.73e-02f  1
2074  3.8527899e+03 3.17e+01 3.24e+04  -3.8 1.79e+03  -3.7 1.73e-03 1.05e-05f  1
2075  3.8317650e+03 3.16e+01 3.36e+04  -3.8 3.51e+03  -4.1 7.84e-03 2.33e-03f  1
2076  3.8317568e+03 3.16e+01 3.36e+04  -3.8 5.35e+03  -4.6 1.94e-06 6.71e-07f  1
2077  3.8261091e+03 3.16e+01 3.35e+04  -3.8 1.45e+04  -5.1 7.06e-05 1.18e-04f  1
2078  3.8258304e+03 3.16e+01 4.36e+04  -3.8 3.66e+02  -2.9 9.48e-02 2.86e-04f  1
2079  3.7306835e+03 3.02e+01 3.51e+04  -3.8 9.43e+02  -3.3 1.41e-05 4.28e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2080  3.6996538e+03 2.94e+01 3.25e+04  -3.8 4.02e+02  -2.9 1.57e-02 2.97e-02f  1
2081  3.6995954e+03 2.94e+01 3.57e+04  -3.8 1.02e+03  -3.4 2.11e-02 2.36e-05f  1
2082  3.6508098e+03 2.81e+01 3.01e+04  -3.8 4.39e+02  -3.0 2.89e-05 4.17e-02f  1
2083  3.5605842e+03 2.39e+01 2.82e+04  -3.8 1.97e+02  -2.5 2.15e-01 1.58e-01f  1
2084  3.4972257e+03 2.26e+01

2188  1.3570743e+03 1.73e-02 7.18e+03  -3.8 2.11e+01  -2.1 6.20e-01 1.00e+00f  1
2189  1.2782600e+03 1.68e-01 3.26e+04  -3.8 6.65e+01  -2.5 3.40e-02 7.65e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2190  1.2718580e+03 1.65e-01 2.79e+04  -3.8 1.77e+02  -3.0 6.54e-02 2.38e-02f  1
2191  1.2706746e+03 1.65e-01 1.56e+04  -3.8 4.23e+02  -3.5 7.78e-02 1.82e-03f  1
2192  1.2428012e+03 1.80e-01 2.18e+04  -3.8 8.36e+02  -4.0 3.97e-05 2.20e-02f  1
2193  1.2385213e+03 1.80e-01 3.42e+03  -3.8 1.23e+03  -4.4 4.95e-02 2.33e-03f  1
2194  1.2290878e+03 1.81e-01 3.33e+03  -3.8 1.61e+03  -4.9 6.85e-06 3.93e-03f  1
2195  1.1390542e+03 4.08e-01 1.85e+04  -3.8 1.60e+03  -5.4 9.21e-03 3.79e-02f  1
2196  1.1390578e+03 4.03e-01 2.97e+04  -3.8 9.52e-02   2.4 6.70e-04 1.29e-02h  1
2197  1.1390584e+03 4.02e-01 3.59e+04  -3.8 9.12e-02   2.0 4.43e-02 3.70e-03h  1
2198  1.1390584e+03 4.02e-01 5.21e+04  -3.8 1.24e-02   1.5 7.88e-01 7.14e-05h  1
2199  1.1391497e+03 3.87e-01

2307  2.8515316e+02 1.88e+00 1.02e+04  -3.8 6.84e+02  -6.8 4.66e-02 4.51e-02f  1
2308  2.8380394e+02 1.87e+00 1.00e+04  -3.8 2.88e+03  -7.3 8.71e-06 2.42e-03f  1
2309  2.5635531e+02 2.81e+00 4.98e+03  -3.8 8.32e+02  -6.8 4.95e-02 1.06e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2310  2.5506466e+02 2.80e+00 4.74e+03  -3.8 4.16e+03  -7.3 2.10e-03 4.67e-03f  1
2311  2.5506298e+02 2.80e+00 5.07e+03  -3.8 2.48e+00   0.5 8.70e-04 1.57e-04h  1
2312  2.5506232e+02 2.80e+00 5.01e+03  -3.8 5.07e-01   1.0 1.78e-05 2.76e-04f  1
2313  2.5506193e+02 2.80e+00 4.99e+03  -3.8 3.59e+01   0.5 1.67e-08 2.57e-06f  1
2314  2.5506190e+02 2.80e+00 4.99e+03  -3.8 1.29e+02   0.9 8.92e-11 4.23e-08f  2
2315  2.5506190e+02 2.80e+00 1.81e+04  -3.8 1.31e-01   3.1 2.91e-02 6.48e-05h  1
2316  2.5506232e+02 2.80e+00 6.64e+04  -3.8 1.26e-01   2.7 1.97e-02 6.30e-04h  2
2317  2.5508252e+02 2.77e+00 7.73e+04  -3.8 4.00e-01   2.2 1.24e-02 8.70e-03h  1
2318  2.5510733e+02 2.75e+00

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2440  7.6074920e+01 1.14e+01 1.21e+03  -3.8 2.33e+02    -  8.41e-02 1.52e-01h  1
2441  7.6077052e+01 9.54e+00 9.05e+01  -3.8 4.39e-02  -0.5 1.00e+00 1.59e-01h  1
2442  7.6071421e+01 8.67e+00 4.11e+01  -3.8 1.50e+02    -  3.00e-01 9.04e-02h  1
2443  7.6071422e+01 8.67e+00 7.62e+02  -3.8 7.21e+00  -0.9 2.44e-01 2.72e-04h  1
2444  7.6198622e+01 4.33e+00 4.64e+02  -3.8 1.67e+02    -  9.74e-02 4.93e-01h  1
2445  7.6165335e+01 4.29e+00 4.08e+02  -3.8 1.54e+00  -0.5 2.79e-02 1.04e-02h  1
2446  7.6092162e+01 4.01e+00 3.13e+02  -3.8 5.25e-01  -0.1 1.40e-01 6.42e-02f  1
2447  7.6055268e+01 2.69e+00 1.80e+02  -3.8 2.53e-01  -0.6 1.00e+00 3.27e-01h  1
2448  7.5954074e+01 7.11e-01 6.84e+02  -3.8 2.92e+02    -  1.21e-01 7.90e-01h  1
2449  7.5962620e+01 1.12e-01 1.36e+02  -3.8 5.68e-02  -1.0 8.05e-01 7.49e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2450  7.5963844e+01 3.07e-03

In [8]:
solve_pyomo(para_truth[4])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 184r 1.4322131e+04 9.64e+04 4.45e+03  -6.7 8.07e+02  -1.3 1.62e-02 6.10e-02f  1
 185r 1.4452785e+04 9.63e+04 4.11e+03  -6.7 3.03e+02  -0.8 3.84e-02 1.45e-01f  1
 186r 1.4452801e+04 9.63e+04 4.08e+03  -6.7 9.07e+02  -1.3 1.41e-02 6.07e-06f  1
 187r 1.4486008e+04 9.63e+04 3.98e+03  -6.7 3.41e+02  -0.9 6.67e-03 3.29e-02f  1
 188r 1.4594116e+04 9.63e+04 3.86e+03  -6.7 1.02e+03  -1.4 1.41e-02 3.58e-02f  1
 189r 1.4977571e+04 9.61e+04 4.53e+03  -6.7 3.83e+02  -0.9 2.99e-02 3.38e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.4977632e+04 9.61e+04 3.50e+03  -6.7 1.15e+03  -1.4 4.51e-02 1.77e-05f  1
 191r 1.5113901e+04 9.61e+04 3.16e+03  -6.7 4.31e+02  -1.0 1.03e-01 1.07e-01f  1
 192r 1.5917811e+04 9.58e+04 5.74e+03  -6.7 1.29e+03  -1.5 8.15e-02 2.11e-01f  1
 193r 1.5917817e+04 9.58e+04 3.51e+03  -6.7 4.84e+02  -1.0 4.65e-01 4.29e-06f  1
 194r 1.6511802e+04 9.56e+04 3.43e+03  -6.7 1.44e+03  -1.5 2.51e-01 1.39e-01f  1
 195r 1.6511803e+04 9.56e+04

 439  1.7461598e+05 2.18e+03 1.18e+04  -1.0 2.18e+03  -0.6 1.42e-03 7.21e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 440  1.7461897e+05 2.17e+03 1.41e+04  -1.0 2.18e+03  -0.1 2.38e-03 1.32e-04h  1
 441  1.7462836e+05 2.17e+03 1.50e+04  -1.0 2.17e+03  -0.6 2.45e-03 4.96e-04h  1
 442  1.7463634e+05 2.17e+03 2.04e+04  -1.0 2.17e+03  -0.2 4.30e-03 3.99e-04h  1
 443  1.7464594e+05 2.17e+03 2.30e+04  -1.0 2.17e+03  -0.7 2.19e-03 6.57e-04h  1
 444  1.7465322e+05 2.17e+03 3.02e+04  -1.0 2.17e+03  -0.2 2.01e-03 4.64e-04h  1
 445  1.7465548e+05 2.17e+03 4.87e+04  -1.0 2.17e+03   0.2 1.63e-03 1.30e-04h  1
 446  1.7466231e+05 2.17e+03 4.82e+04  -1.0 2.17e+03  -0.3 3.65e-04 4.66e-04h  1
 447  1.7466287e+05 2.17e+03 4.78e+04  -1.0 2.17e+03   0.1 5.85e-06 3.57e-05h  1
 448  1.7466313e+05 2.17e+03 7.54e+04  -1.0 2.17e+03   0.6 9.44e-04 1.42e-05h  1
 449  1.7468477e+05 2.17e+03 5.77e+04  -1.0 2.17e+03   0.1 5.28e-05 1.47e-03h  1
iter    objective    inf_pr 

 559  1.7751420e+05 1.29e+03 5.11e+07  -1.0 1.93e+03  -0.9 1.27e-03 2.98e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 560  1.7751500e+05 1.29e+03 5.27e+07  -1.0 1.74e+03  -0.5 8.47e-02 2.14e-04h  1
 561  1.7751860e+05 1.29e+03 5.24e+07  -1.0 1.95e+03  -1.0 8.97e-03 1.04e-03h  1
 562  1.7752822e+05 1.28e+03 5.24e+07  -1.0 1.89e+03  -0.6 9.00e-03 2.75e-03h  1
 563  1.7753223e+05 1.28e+03 5.01e+07  -1.0 2.07e+03  -1.0 6.35e-02 1.08e-03h  1
 564  1.7764841e+05 1.24e+03 4.81e+07  -1.0 2.00e+03  -0.6 3.46e-04 3.07e-02h  1
 565  1.7779855e+05 1.19e+03 4.77e+07  -1.0 1.94e+03  -1.1 9.18e-05 4.02e-02h  1
 566  1.7779938e+05 1.19e+03 5.06e+07  -1.0 1.76e+03  -0.7 1.82e-01 2.38e-04h  1
 567  1.7780703e+05 1.19e+03 5.01e+07  -1.0 1.88e+03  -1.1 2.54e-02 2.31e-03h  1
 568  1.7781502e+05 1.19e+03 5.01e+07  -1.0 1.97e+03  -0.7 4.99e-03 2.72e-03h  1
 569  1.7781499e+05 1.19e+03 5.37e+07  -1.0 1.71e+03  -1.2 1.23e-02 4.61e-05h  1
iter    objective    inf_pr 

 681  1.8058333e+05 3.49e+01 9.95e+06  -1.0 8.16e+02  -2.6 6.67e-03 1.36e-02h  1
 682  1.8058349e+05 3.49e+01 9.91e+06  -1.0 9.90e+02  -0.4 1.78e-04 3.57e-04h  1
 683  1.8058361e+05 3.48e+01 1.20e+07  -1.0 1.18e+03  -0.9 4.48e-02 5.39e-04h  1
 684  1.8058800e+05 3.44e+01 1.18e+07  -1.0 9.25e+02  -1.3 6.19e-03 1.14e-02h  1
 685  1.8058821e+05 3.44e+01 1.18e+07  -1.0 1.11e+03  -0.9 3.81e-05 8.60e-04h  1
 686  1.8058925e+05 3.43e+01 1.22e+07  -1.0 9.26e+02  -1.4 1.08e-01 3.14e-03h  1
 687  1.8059909e+05 3.27e+01 9.30e+06  -1.0 9.06e+02  -1.9 3.17e-01 4.55e-02h  1
 688  1.8059914e+05 3.23e+01 7.72e+06  -1.0 7.55e+02  -2.3 1.50e-01 1.23e-02h  1
 689  1.8060727e+05 3.07e+01 5.73e+06  -1.0 8.50e+02  -1.9 3.52e-01 5.11e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 690  1.8060343e+05 2.89e+01 5.78e+06  -1.0 6.88e+02  -2.4 8.01e-03 5.98e-02f  1
 691  1.8060350e+05 2.88e+01 5.61e+06  -1.0 1.02e+03  -0.2 1.55e-04 8.41e-04h  1
 692  1.8060372e+05 2.88e+01

...

1039  6.7114245e+04 4.90e+01 4.62e+03  -2.5 2.28e+03  -3.6 7.21e-04 1.76e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1040  6.7094099e+04 4.90e+01 4.51e+03  -2.5 6.65e+03  -4.1 1.82e-02 1.08e-03f  1
1041  6.7093795e+04 4.90e+01 4.23e+03  -2.5 3.52e+02  -2.8 4.77e-01 4.79e-04f  1
1042  6.6282074e+04 3.23e+01 2.56e+03  -2.5 9.85e+02  -3.3 4.28e-01 3.40e-01f  1
1043  6.5473898e+04 2.90e+01 2.65e+03  -2.5 2.86e+03  -3.7 2.61e-03 1.04e-01f  1
1044  6.5388252e+04 2.89e+01 1.82e+03  -2.5 8.39e+03  -4.2 1.39e-01 3.58e-03f  1
1045  6.5372141e+04 2.88e+01 1.82e+03  -2.5 2.67e+04  -4.7 2.31e-04 1.99e-04f  1
1046  6.4940574e+04 2.88e+01 1.82e+03  -2.5 3.41e+05  -5.2 6.39e-07 5.38e-04f  1
1047  6.2471519e+04 2.82e+01 1.77e+03  -2.5 4.44e+04  -4.8 2.71e-02 2.27e-02f  1
1048  6.2417645e+04 2.82e+01 1.79e+03  -2.5 9.49e+05  -5.2 9.50e-04 2.50e-05f  1
1049  5.9689512e+04 2.85e+01 1.75e+03  -2.5 5.27e+04  -4.8 2.63e-02 2.10e-02f  1
iter    objective    inf_pr 

1168  3.7625456e+03 3.63e+01 4.27e+03  -2.5 1.08e+04  -4.0 1.29e-06 3.07e-03f  1
1169  3.7621081e+03 3.63e+01 4.27e+03  -2.5 1.87e+04  -4.4 9.21e-04 5.14e-06f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1170  3.7042531e+03 3.63e+01 6.70e+03  -2.5 1.01e+04  -4.9 5.74e-02 2.07e-03f  1
1171  3.7040714e+03 3.63e+01 6.71e+03  -2.5 7.48e+03  -5.4 1.52e-04 9.52e-06f  1
1172  3.6948184e+03 3.62e+01 6.64e+03  -2.5 3.54e+03  -5.0 4.70e-05 9.49e-04f  1
1173  3.6943140e+03 3.62e+01 6.86e+03  -2.5 3.37e+03  -5.5 2.89e-03 1.40e-04f  1
1174  3.6761812e+03 3.61e+01 6.50e+03  -2.5 2.59e+03  -5.0 8.31e-06 4.23e-03f  1
1175  3.4101342e+03 3.60e+01 7.09e+03  -2.5 2.87e+04  -4.6 7.75e-04 3.66e-03f  1
1176  3.4093765e+03 3.59e+01 7.09e+03  -2.5 2.75e+04  -5.1 3.12e-06 1.17e-05f  1
1177  3.4093147e+03 3.59e+01 7.14e+03  -2.5 5.16e+03  -5.6 6.32e-04 4.84e-06f  1
1178  3.4017952e+03 3.59e+01 1.02e+04  -2.5 2.11e+03  -5.1 4.62e-02 2.50e-03f  1
1179  3.3719726e+03 3.55e+01

1292  1.0046992e+03 2.90e+00 6.67e+03  -2.5 3.76e+02  -1.8 2.37e-03 8.32e-04f  1
1293  1.0045737e+03 2.90e+00 6.71e+03  -2.5 3.78e+01  -1.4 1.22e-02 3.43e-03f  1
1294  1.0012207e+03 2.84e+00 7.24e+03  -2.5 3.36e+01  -1.9 7.69e-02 2.20e-02f  1
1295  9.8928638e+02 2.84e+00 6.68e+03  -2.5 7.50e+01  -2.4 1.54e-02 3.42e-02f  1
1296  9.8670959e+02 2.77e+00 6.46e+03  -2.5 9.59e+00  -1.0 4.27e-02 6.14e-02f  1
1297  8.6668496e+02 3.10e+00 6.45e+03  -2.5 1.12e+05    -  3.48e-04 1.06e-03f  1
1298  8.6664668e+02 3.09e+00 5.86e+03  -2.5 2.98e+01  -1.5 4.86e-02 2.67e-04f  1
1299  8.6323947e+02 3.10e+00 5.88e+03  -2.5 4.84e+04    -  5.48e-04 7.15e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1300  8.6321465e+02 3.09e+00 4.68e+03  -2.5 8.45e+00  -1.1 4.50e-01 5.72e-04f  1
1301  8.5964091e+02 2.95e+00 7.14e+03  -2.5 1.53e+01  -1.6 3.45e-01 4.78e-02f  1
1302  8.5787217e+02 2.95e+00 7.15e+03  -2.5 2.76e+04    -  1.67e-03 5.96e-05f  1
1303  5.6331937e+02 6.39e+00

1425  9.2527334e+01 5.64e+00 8.77e+04  -3.8 5.04e+01    -  1.22e-01 3.24e-01h  1
1426  9.2528065e+01 5.60e+00 8.69e+04  -3.8 1.44e-02   0.8 1.00e+00 8.72e-03h  1
1427  9.2407008e+01 4.38e+00 6.81e+04  -3.8 2.99e+01    -  1.72e-01 2.17e-01h  1
1428  9.2407281e+01 4.35e+00 6.77e+04  -3.8 3.79e-02   0.4 1.00e+00 5.41e-03h  1
1429  9.2026468e+01 2.21e+00 3.45e+04  -3.8 1.07e+02    -  8.25e-02 4.91e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1430  9.2037489e+01 1.51e-06 6.64e-02  -3.8 1.08e-02   0.8 1.00e+00 1.00e+00h  1
1431  9.0811166e+01 1.07e-01 5.52e+02  -3.8 4.73e+02    -  1.32e-02 1.45e-01f  1
1432  9.0799143e+01 1.12e-01 1.09e+03  -3.8 1.11e+00   0.3 6.22e-01 1.79e-02h  1
1433  9.0524772e+01 1.72e-01 9.98e+02  -3.8 1.55e+04    -  9.20e-05 5.21e-03f  1
1434  9.0525589e+01 1.57e-01 1.69e+02  -3.8 1.32e-01  -0.2 1.00e+00 8.82e-02f  4
1435  9.0039287e+01 3.04e-01 2.66e+02  -3.8 3.54e+03    -  2.23e-02 1.56e-02f  1
1436  8.8839130e+01 1.28e+00

1553  7.9712631e+01 2.31e+00 3.40e+02  -5.7 4.82e+04    -  2.36e-09 1.10e-05f  1
1554  7.9712627e+01 2.31e+00 3.42e+02  -5.7 2.20e+04    -  1.15e-08 1.12e-05f  1
1555  7.9712627e+01 2.31e+00 3.42e+02  -5.7 9.83e+03    -  5.53e-08 5.62e-06f  2
1556  7.9712629e+01 2.30e+00 4.37e+02  -5.7 7.79e-02  -0.8 1.00e+00 3.20e-03h  1
1557  7.9712469e+01 1.42e+00 2.24e+01  -5.7 1.02e+01    -  1.00e+00 3.86e-01h  1
1558  7.9712685e+01 2.05e-06 5.74e-03  -5.7 7.40e+00    -  1.00e+00 1.00e+00h  1
1559  7.9712685e+01 1.46e-11 5.42e-09  -5.7 2.71e-03    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1560  7.9712232e+01 3.30e-06 4.19e-01  -8.6 1.33e+00    -  9.96e-01 9.93e-01f  1
1561  7.9712229e+01 1.84e-11 6.71e-08  -8.6 7.63e-03    -  1.00e+00 1.00e+00h  1
1562  7.9712229e+01 1.46e-11 7.24e-13  -8.6 1.49e-07    -  1.00e+00 1.00e+00h  1

Number of Iterations....: 1562

                                   (scaled)                 (unscaled)
Objec

In [9]:
solve_pyomo(para_truth[5])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 187r 1.1969459e+04 9.71e+04 3.45e-01  -3.0 1.92e+00  -0.7 1.00e+00 1.00e+00f  1
 188r 1.1992447e+04 9.71e+04 1.79e+02  -3.0 3.00e+01    -  1.05e-01 1.67e-01f  1
 189r 1.1994942e+04 9.71e+04 2.80e-01  -3.0 5.83e-01  -0.3 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.2001928e+04 9.71e+04 2.58e-01  -3.0 1.61e+00  -0.8 1.00e+00 1.00e+00f  1
 191r 1.2004481e+04 9.71e+04 2.51e-01  -3.0 5.87e-01  -0.4 1.00e+00 1.00e+00f  1
 192r 1.2011578e+04 9.71e+04 2.30e-01  -3.0 1.62e+00  -0.8 1.00e+00 1.00e+00f  1
 193r 1.2014162e+04 9.71e+04 2.23e-01  -3.0 5.86e-01  -0.4 1.00e+00 1.00e+00f  1
 194r 1.2017087e+04 9.71e+04 1.05e+02  -3.0 1.60e+00  -0.9 1.00e+00 4.10e-01f  1
 195r 1.2019827e+04 9.71e+04 2.06e-01  -3.0 6.12e-01  -0.5 1.00e+00 1.00e+00f  1
 196r 1.2024935e+04 9.71e+04 8.62e+01  -3.0 1.65e+00  -0.9 2.04e-01 6.77e-01f  1
 197r 1.2059313e+04 9.71e+04 6.63e+01  -3.0 1.76e+01    -  4.91e-01 4.21e-01f  1
 198r 1.2063383e+04 9.71e+04

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 480  4.1245345e+05 7.29e+02 1.55e+05  -1.0 2.65e+04  -2.8 3.68e-05 2.70e-03f  1
 481  4.1261702e+05 7.03e+02 1.21e+05  -1.0 2.65e+04  -3.3 4.61e-03 3.55e-02h  2
 482  4.1275925e+05 6.88e+02 2.46e+05  -1.0 2.64e+04  -2.8 6.06e-02 2.14e-02h  1
 483  4.1282443e+05 6.82e+02 6.69e+05  -1.0 2.64e+04  -2.4 5.58e-02 8.46e-03h  1
 484  4.1283549e+05 6.81e+02 1.03e+06  -1.0 2.64e+04  -2.9 1.73e-01 1.84e-03h  1
 485  4.1283209e+05 6.80e+02 1.08e+06  -1.0 2.66e+04  -2.5 7.77e-03 6.94e-04f  1
 486  4.1277843e+05 6.73e+02 1.06e+06  -1.0 2.67e+04  -2.9 2.69e-04 1.04e-02f  1
 487  4.1300061e+05 6.48e+02 7.99e+05  -1.0 2.67e+04  -2.5 8.70e-04 3.81e-02h  1
 488  4.1303696e+05 6.41e+02 9.75e+05  -1.0 2.66e+04  -3.0 1.96e-01 1.09e-02h  1
 489  4.1311554e+05 6.31e+02 1.18e+06  -1.0 2.66e+04  -2.6 5.74e-02 1.45e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 490  4.1312608e+05 6.30e+02

 603  3.6306784e+05 5.51e+02 9.32e+04  -1.0 1.50e+04  -4.6 1.60e-01 3.91e-04f  1
 604  3.6303870e+05 5.41e+02 8.07e+04  -1.0 1.44e+04  -5.1 7.45e-02 1.84e-02f  1
 605  3.6259747e+05 4.67e+02 5.88e+04  -1.0 1.42e+04  -4.6 2.11e-01 1.64e-01f  1
 606  3.6259779e+05 4.66e+02 6.04e+04  -1.0 1.22e+04  -3.3 2.55e-02 2.32e-04h  1
 607  3.6243753e+05 4.28e+02 5.99e+04  -1.0 1.27e+04  -3.8 7.27e-04 8.89e-02f  1
 608  3.6244772e+05 4.28e+02 6.07e+04  -1.0 1.01e+04  -2.4 2.19e-03 1.24e-03h  1
 609  3.6245108e+05 4.27e+02 8.27e+04  -1.0 9.93e+03  -2.0 8.02e-03 4.16e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 610  3.6273997e+05 4.13e+02 4.93e+04  -1.0 1.00e+04  -2.5 3.09e-04 3.43e-02h  1
 611  3.6274122e+05 4.13e+02 6.01e+04  -1.0 9.20e+03  -2.1 3.98e-03 1.48e-04h  1
 612  3.6276979e+05 4.12e+02 4.74e+04  -1.0 9.42e+03  -1.6 1.39e-03 3.22e-03h  1
 613  3.6277806e+05 4.11e+02 4.90e+04  -1.0 9.38e+03  -2.1 1.70e-03 9.35e-04h  1
 614  3.6278117e+05 4.11e+02

 724  3.5933853e+05 2.96e+01 3.84e+04  -1.0 2.46e+03  -4.5 1.42e-01 9.11e-04f  1
 725  3.5821703e+05 2.92e+01 2.42e+05  -1.0 4.08e+03  -5.0 6.27e-04 9.01e-02f  1
 726  3.5777357e+05 2.91e+01 2.19e+05  -1.0 2.77e+04  -5.5 1.28e-02 5.42e-03f  1
 727  3.5776804e+05 2.91e+01 9.11e+04  -1.0 4.49e+03  -5.0 4.53e-02 4.35e-04f  1
 728  3.5756520e+05 2.89e+01 1.13e+05  -1.0 1.24e+04  -5.5 4.34e-04 8.53e-03f  1
 729  3.5633574e+05 2.85e+01 1.94e+05  -1.0 1.07e+04  -5.1 8.71e-03 3.91e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 730  3.5632377e+05 2.85e+01 1.82e+05  -1.0 2.19e+04  -5.6 4.31e-03 1.93e-04f  1
 731  3.5631589e+05 2.85e+01 1.12e+05  -1.0 4.72e+03  -5.1 2.61e-02 1.75e-03f  1
 732  3.5570448e+05 2.65e+01 3.20e+05  -1.0 2.57e+03  -4.7 6.80e-04 7.72e-02f  1
 733  3.5485680e+05 2.65e+01 3.50e+05  -1.0 1.39e+04  -5.2 9.31e-03 2.16e-02f  1
 734  3.5484809e+05 2.64e+01 2.88e+04  -1.0 4.41e+03  -4.8 1.08e-01 6.87e-04f  1
 735  3.5477067e+05 2.62e+01

...

2062  3.9935407e+03 5.63e+00 1.29e+03  -3.8 3.22e+05  -5.8 1.53e-04 6.15e-05f  1
2063  3.7690098e+03 6.15e+00 1.28e+03  -3.8 2.27e+06  -6.3 1.11e-07 2.39e-05f  1
2064  3.7561698e+03 6.15e+00 1.57e+03  -3.8 4.04e+05  -5.9 5.18e-04 7.79e-06f  1
2065  3.5258159e+03 6.81e+00 1.57e+03  -3.8 1.10e+05  -5.5 4.89e-04 4.92e-04f  1
2066  3.5058903e+03 6.81e+00 1.51e+03  -3.8 5.01e+03  -4.1 1.78e-06 9.28e-04f  1
2067  3.5056181e+03 6.81e+00 1.58e+03  -3.8 1.32e+03  -2.8 8.27e-04 8.84e-05f  1
2068  3.5023557e+03 6.80e+00 1.50e+03  -3.8 9.33e+02  -3.3 4.09e-06 1.23e-03f  1
2069  3.5019969e+03 6.80e+00 1.51e+03  -3.8 1.74e+03  -2.9 1.89e-04 9.36e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2070  3.5019941e+03 6.80e+00 1.51e+03  -3.8 2.14e+03  -3.3 2.37e-08 8.10e-07f  1
2071  3.5019915e+03 6.80e+00 1.86e+03  -3.8 4.73e+03  -2.9 1.32e-03 2.96e-07f  1
2072  3.4952204e+03 6.79e+00 1.78e+03  -3.8 9.77e+02  -3.4 3.89e-04 1.86e-03f  1
2073  3.4952173e+03 6.79e+00

2179  1.6124231e+03 1.80e+01 4.69e+04  -3.8 1.48e+01  -1.2 1.87e-02 6.82e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2180  1.6099059e+03 1.75e+01 4.59e+04  -3.8 2.49e+01  -1.6 2.16e-02 2.65e-02f  1
2181  1.6073268e+03 1.73e+01 4.55e+04  -3.8 6.18e+01  -2.1 7.15e-03 1.02e-02f  1
2182  1.6073262e+03 1.73e+01 4.54e+04  -3.8 1.06e+01  -0.8 5.01e-04 2.18e-05h  1
2183  1.6072221e+03 1.73e+01 4.51e+04  -3.8 1.07e+02  -1.3 3.61e-04 2.87e-03f  1
2184  1.6072215e+03 1.73e+01 4.50e+04  -3.8 2.53e+01  -0.8 1.69e-02 2.65e-05h  1
2185  1.5784811e+03 9.33e+00 5.45e+04  -3.8 1.54e+01  -1.3 2.63e-02 4.73e-01f  1
2186  1.5713474e+03 8.94e+00 5.50e+04  -3.8 3.56e+01  -1.8 2.90e-03 4.30e-02f  1
2187  1.5700143e+03 8.15e+00 5.98e+04  -3.8 4.67e+00  -0.5 4.04e-02 8.83e-02f  1
2188  1.5640599e+03 6.47e+00 2.66e+04  -3.8 6.06e+00  -0.9 4.23e-01 2.08e-01f  1
2189  1.5640109e+03 6.46e+00 3.28e+04  -3.8 1.50e+01  -1.4 2.03e-01 8.12e-04h  1
iter    objective    inf_pr 

2304  3.6238762e+02 1.53e+00 6.76e+03  -3.8 3.96e+01  -2.0 5.42e-05 1.52e-02f  1
2305  3.6193798e+02 1.48e+00 6.64e+03  -3.8 1.60e+01  -2.4 3.82e-03 2.88e-02f  1
2306  3.6189950e+02 1.46e+00 5.64e+03  -3.8 1.30e+00  -1.1 3.11e-01 1.33e-02h  1
2307  3.6185724e+02 1.45e+00 5.58e+03  -3.8 2.96e+00  -1.6 2.67e-02 1.04e-02h  1
2308  3.6082774e+02 1.26e+00 4.81e+03  -3.8 7.62e+00  -2.1 1.97e-02 1.43e-01f  1
2309  3.6058926e+02 1.18e+00 4.54e+03  -3.8 3.32e+00  -1.6 3.61e-02 6.60e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2310  3.6045331e+02 1.11e+00 4.36e+03  -3.8 1.47e+00  -1.2 3.07e-02 5.47e-02h  1
2311  3.6011108e+02 1.02e+00 3.82e+03  -3.8 3.67e+00  -1.7 1.84e-01 8.09e-02f  1
2312  3.5985531e+02 9.07e-01 2.80e+03  -3.8 1.62e+00  -1.3 2.87e-01 1.11e-01h  1
2313  3.5922260e+02 7.65e-01 2.25e+03  -3.8 4.06e+00  -1.7 2.76e-01 1.53e-01f  1
2314  3.5810662e+02 3.38e-01 3.49e+02  -3.8 1.69e+00  -1.3 1.00e+00 5.51e-01f  1
2315  3.5800526e+02 3.28e-01

2437  7.7753537e+01 5.83e+00 5.31e+02  -5.7 3.85e+02  -6.4 6.20e-02 1.17e-02h  1
2438  7.7751667e+01 5.83e+00 5.60e+02  -5.7 6.15e+02  -6.9 8.95e-03 2.00e-04h  1
2439  7.7751775e+01 5.82e+00 8.60e+04  -5.7 3.69e-02   3.0 3.51e-01 4.17e-04h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2440  7.7751828e+01 5.82e+00 1.03e+05  -5.7 5.63e-02   2.5 9.12e-01 1.92e-04h  1
2441  7.7765691e+01 5.54e+00 4.50e+04  -5.7 7.89e-02   2.0 1.00e+00 4.86e-02h  1
2442  7.7699484e+01 5.50e+00 3.58e+04  -5.7 7.88e+02    -  1.97e-01 7.26e-03h  1
2443  7.7707091e+01 5.35e+00 3.54e+04  -5.7 8.09e-02   1.5 1.48e-05 2.84e-02h  1
2444  7.7708826e+01 5.31e+00 2.82e+03  -5.7 8.00e-02   1.1 1.00e+00 6.65e-03h  4
2445  7.6880339e+01 5.34e+00 2.40e+03  -5.7 5.27e+02    -  1.24e-01 9.61e-02f  1
2446  7.6880302e+01 5.34e+00 2.40e+03  -5.7 8.92e+02    -  4.17e-07 2.74e-06h  1
2447  7.6880288e+01 5.34e+00 2.40e+03  -5.7 6.12e+02    -  1.40e-06 1.52e-06f  1
2448  7.6880313e+01 5.34e+00

2573  7.0589462e+01 9.77e-04 6.75e-02  -5.7 5.06e-03  -0.5 1.00e+00 1.00e+00h  1
2574  7.0589345e+01 2.89e-01 3.95e+00  -5.7 3.85e-02  -1.0 1.00e+00 1.00e+00h  1
2575  7.0589126e+01 6.90e-02 2.49e+00  -5.7 1.96e-02  -0.6 1.00e+00 9.58e-01h  1
2576  7.0579107e+01 2.30e+00 4.18e+02  -5.7 6.50e+02    -  3.75e-03 1.39e-01f  1
2577  7.0579019e+01 2.29e+00 4.17e+02  -5.7 1.04e+03    -  9.72e-05 8.51e-04h  1
2578  7.0579016e+01 2.29e+00 4.17e+02  -5.7 1.49e+04    -  3.37e-08 1.69e-06f  2
2579  7.0579016e+01 2.29e+00 1.30e+03  -5.7 1.24e+00  -1.0 4.22e-01 5.35e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2580  7.0578323e+01 1.47e+00 1.25e+03  -5.7 3.06e+01    -  3.38e-02 3.58e-01f  1
2581  7.0578359e+01 1.17e+00 2.73e+00  -5.7 1.21e-02  -1.5 1.00e+00 2.06e-01h  1
2582  7.0576825e+01 5.32e-01 7.22e+01  -5.7 4.88e+01    -  3.23e-02 5.39e-01h  1
2583  7.0576759e+01 3.44e-01 1.19e+02  -5.7 1.55e-01  -2.0 1.00e+00 3.61e-01f  1
2584  7.0564644e+01 6.04e+00

In [10]:
solve_pyomo(para_truth[6])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 298r 3.2149006e+04 9.04e+04 9.05e+03  -6.7 2.16e+02  -0.8 2.79e-01 3.33e-03f  1
In iteration 298, 1 Slack too small, adjusting variable bound
 299r 3.2303320e+04 9.04e+04 8.36e+03  -6.7 6.46e+02  -1.2 9.03e-02 6.57e-02f  1
In iteration 299, 1 Slack too small, adjusting variable bound
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 300r 3.2303321e+04 9.04e+04 1.23e+04  -6.7 2.43e+02  -0.8 1.00e+00 3.14e-07f  1
In iteration 300, 1 Slack too small, adjusting variable bound
 301r 3.2537562e+04 9.03e+04 1.22e+04  -6.7 7.21e+02  -1.3 1.65e-01 8.81e-02f  1
In iteration 301, 1 Slack too small, adjusting variable bound
 302r 3.2537562e+04 9.03e+04 1.46e+04  -6.7 2.72e+02  -0.9 1.00e+00 9.74e-08f  1
In iteration 302, 1 Slack too small, adjusting variable bound
 303r 3.2622254e+04 9.03e+04 1.61e+04  -6.7 8.07e+02  -1.3 1.56e-01 3.03e-02f  1
In iteration 303, 1 Slack too small, adjusting variable bound
 304r 3.2622257e+04 9.03e+04 1.75e+04  -6.7 3.05e+02  -0.9 1.

 452  1.7572869e+05 7.04e+01 5.21e+05  -1.0 3.39e+02  -0.9 1.83e-03 2.39e-02h  1
 453  1.7572931e+05 7.03e+01 6.38e+05  -1.0 3.27e+02  -0.4 8.84e-02 1.51e-03h  1
 454  1.7573322e+05 6.83e+01 6.19e+05  -1.0 3.15e+02  -0.9 4.45e-03 2.95e-02h  1
 455  1.7573344e+05 6.82e+01 7.33e+05  -1.0 2.98e+02  -0.5 9.95e-02 1.39e-03h  1
 456  1.7573248e+05 6.65e+01 7.19e+05  -1.0 3.22e+02  -1.0 1.78e-03 2.38e-02h  1
 457  1.7573252e+05 6.63e+01 8.12e+05  -1.0 2.99e+02  -0.5 1.09e-01 3.58e-03h  1
 458  1.7573037e+05 6.48e+01 7.98e+05  -1.0 3.11e+02  -1.0 6.27e-03 2.20e-02h  1
 459  1.7573009e+05 6.43e+01 8.41e+05  -1.0 3.36e+02  -0.6 7.14e-02 8.23e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 460  1.7572834e+05 6.32e+01 8.32e+05  -1.0 3.48e+02  -1.1 2.06e-03 1.68e-02h  1
 461  1.7572489e+05 6.22e+01 8.06e+05  -1.0 4.31e+02  -1.5 3.24e-02 1.60e-02h  1
 462  1.7572485e+05 6.21e+01 7.95e+05  -1.0 3.98e+02  -1.1 2.60e-02 2.48e-03h  1
 463  1.7572550e+05 6.16e+01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 570  1.7576514e+05 1.28e+01 1.68e+06  -1.0 3.48e+02  -1.6 1.93e-02 3.58e-03h  1
 571  1.7576073e+05 1.27e+01 1.69e+06  -1.0 8.08e+02  -2.1 3.28e-03 2.14e-02f  1
 572  1.7575787e+05 1.26e+01 1.63e+06  -1.0 3.11e+02  -2.5 3.02e-02 4.46e-03f  1
 573  1.7570469e+05 1.22e+01 1.25e+06  -1.0 6.64e+02  -3.0 1.72e-01 3.07e-02f  1
 574  1.7564886e+05 1.21e+01 1.20e+06  -1.0 1.06e+03  -3.5 2.91e-02 1.21e-02f  1
 575  1.7562023e+05 1.19e+01 1.08e+06  -1.0 3.14e+02  -3.1 6.75e-02 1.67e-02f  1
 576  1.7560699e+05 1.19e+01 1.02e+06  -1.0 2.24e+03  -3.6 4.01e-02 2.52e-03f  1
 577  1.7555449e+05 1.16e+01 1.00e+06  -1.0 1.69e+03  -3.1 1.19e-03 1.87e-02f  1
 578  1.7539713e+05 1.14e+01 9.10e+05  -1.0 2.63e+03  -3.6 5.91e-02 2.40e-02f  1
 579  1.7538467e+05 1.13e+01 9.09e+05  -1.0 1.39e+03  -3.2 9.33e-04 3.84e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 580  1.7532695e+05 1.11e+01

 691  1.0763849e+05 5.17e+00 6.95e+05  -1.0 2.92e+01  -1.5 3.38e-01 7.66e-01f  1
 692  1.0757320e+05 3.82e+00 5.46e+04  -1.0 6.87e+01  -2.0 9.33e-01 2.65e-01f  1
 693  1.0735867e+05 2.64e+00 5.76e+04  -1.0 1.94e+02  -2.5 6.67e-01 3.13e-01f  1
 694  1.0708346e+05 2.29e+00 4.75e+04  -1.0 5.71e+02  -3.0 1.57e-02 1.36e-01f  1
 695  1.0698249e+05 2.26e+00 4.86e+04  -1.0 1.76e+03  -3.4 1.15e-01 1.59e-02f  1
 696  1.0697443e+05 2.25e+00 5.59e+04  -1.0 6.46e+02  -3.0 4.58e-01 3.51e-03f  1
 697  1.0664840e+05 2.13e+00 5.33e+04  -1.0 1.83e+03  -3.5 1.12e-01 5.49e-02f  1
 698  1.0625160e+05 2.08e+00 5.25e+04  -1.0 5.56e+03  -4.0 6.90e-02 2.26e-02f  1
 699  1.0619441e+05 2.08e+00 5.25e+04  -1.0 1.72e+04  -4.4 1.91e-02 1.16e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 700  1.0470656e+05 2.75e+00 5.21e+04  -1.0 6.81e+04  -4.9 9.75e-03 8.56e-03f  1
 701  1.0416402e+05 2.88e+00 5.21e+04  -1.0 1.32e+06  -5.4 1.28e-04 1.91e-04f  1
 702  1.0413060e+05 2.88e+00

...

1028  3.1292909e+03 1.37e+01 9.63e+03  -3.8 1.97e+03  -5.2 1.54e-05 8.85e-05f  1
1029  3.1264556e+03 1.37e+01 9.64e+03  -3.8 8.76e+03  -5.7 2.12e-04 1.43e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1030  3.1261965e+03 1.37e+01 9.64e+03  -3.8 1.99e+03  -5.2 1.75e-05 9.76e-05f  1
1031  3.1235653e+03 1.37e+01 9.65e+03  -3.8 9.84e+03  -5.7 2.11e-04 1.36e-04f  1
1032  3.1232542e+03 1.37e+01 9.65e+03  -3.8 2.00e+03  -5.3 2.04e-05 1.15e-04f  1
1033  3.1207915e+03 1.37e+01 9.67e+03  -3.8 1.13e+04  -5.8 2.15e-04 1.29e-04f  1
1034  3.1203833e+03 1.37e+01 9.66e+03  -3.8 2.05e+03  -5.3 2.40e-05 1.41e-04f  1
1035  3.1180104e+03 1.37e+01 9.65e+03  -3.8 1.58e+04  -5.8 5.90e-05 9.92e-05f  1
1036  3.1170145e+03 1.37e+01 9.66e+03  -3.8 7.94e+03  -3.6 5.02e-06 3.47e-05f  1
1037  3.1103586e+03 1.36e+01 9.69e+03  -3.8 6.86e+02  -3.2 7.79e-05 2.69e-03f  1
1038  3.1088997e+03 1.36e+01 9.58e+03  -3.8 1.53e+04  -3.6 1.57e-04 2.54e-05f  1
1039  3.0912220e+03 1.36e+01

1143  1.3223837e+03 2.98e+00 6.38e+04  -3.8 1.02e+02  -2.1 2.59e-02 2.17e-03f  1
1144  1.3220149e+03 2.97e+00 6.36e+04  -3.8 3.22e+01  -1.6 6.36e-03 2.93e-03f  1
1145  1.3212686e+03 2.97e+00 6.33e+04  -3.8 9.41e+01  -2.1 1.19e-02 2.00e-03f  1
1146  1.3135936e+03 2.95e+00 6.29e+04  -3.8 3.08e+02  -2.6 1.54e-02 6.14e-03f  1
1147  1.3135254e+03 2.95e+00 6.34e+04  -3.8 1.14e+03  -3.1 2.98e-02 2.04e-05f  1
1148  1.3122349e+03 2.95e+00 6.34e+04  -3.8 6.56e+03  -3.5 4.47e-06 7.50e-05f  1
1149  1.3119403e+03 2.95e+00 6.58e+04  -3.8 8.46e+02  -4.0 2.18e-02 1.41e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1150  1.3094102e+03 2.92e+00 6.43e+04  -3.8 1.41e+02  -4.5 1.14e-03 1.13e-02f  1
1151  1.3072497e+03 2.89e+00 6.38e+04  -3.8 4.40e+02  -5.0 6.54e-03 7.19e-03f  1
1152  1.2925975e+03 2.89e+00 6.34e+04  -3.8 2.25e+04  -5.4 1.26e-05 5.25e-04f  1
1153  1.2922924e+03 2.89e+00 6.35e+04  -3.8 2.82e+03  -4.1 1.21e-03 8.48e-05f  1
1154  1.2919093e+03 2.89e+00

1261  3.7519089e+02 2.68e+00 2.13e+04  -3.8 9.37e+03  -5.1 1.40e-03 2.89e-04f  1
1262  3.7516049e+02 2.68e+00 2.13e+04  -3.8 6.39e+04  -5.6 2.22e-06 1.08e-06f  1
1263  3.7356778e+02 2.68e+00 2.13e+04  -3.8 5.85e+03  -5.1 2.19e-03 6.18e-04f  1
1264  3.7331902e+02 2.68e+00 2.13e+04  -3.8 1.14e+03  -5.6 1.59e-05 4.06e-04f  1
1265  3.7133939e+02 2.68e+00 2.14e+04  -3.8 1.29e+04  -5.2 1.18e-03 3.71e-04f  1
1266  3.7121482e+02 2.68e+00 2.14e+04  -3.8 1.18e+03  -5.7 1.58e-05 1.98e-04f  1
1267  3.6218284e+02 2.57e+00 2.03e+04  -3.8 4.46e+02  -4.3 5.91e-02 4.04e-02f  1
1268  3.6212905e+02 2.57e+00 2.03e+04  -3.8 7.52e+02  -4.8 1.74e-05 1.38e-04f  1
1269  3.5403206e+02 2.48e+00 1.94e+04  -3.8 5.14e+02  -4.4 5.54e-02 3.37e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1270  3.5398965e+02 2.48e+00 1.94e+04  -3.8 7.61e+02  -4.9 1.86e-05 1.09e-04f  1
1271  3.4616066e+02 2.40e+00 1.86e+04  -3.8 6.15e+02  -4.4 8.05e-02 2.93e-02f  1
1272  3.4610235e+02 2.40e+00

1387  8.5642358e+01 1.36e-04 5.42e+00  -3.8 1.09e-02   1.6 1.00e+00 9.74e-01f  1
1388  8.5624179e+01 2.83e-07 3.96e-01  -3.8 3.47e-02   1.1 1.00e+00 1.00e+00F  1
1389  8.5613867e+01 3.09e-03 4.89e-01  -3.8 1.54e-02   1.5 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1390  8.5578391e+01 1.65e-02 1.04e+02  -3.8 7.45e-02   1.0 1.00e+00 5.64e-01H  1
1391  8.5565238e+01 5.18e-03 4.48e+00  -3.8 2.23e-02   1.5 1.00e+00 8.34e-01f  1
1392  8.5560641e+01 5.20e-03 2.08e+02  -3.8 7.05e-02   1.0 1.00e+00 5.75e-02f  1
1393  8.4593299e+01 3.51e-01 1.97e+02  -3.8 6.40e+03    -  1.36e-02 2.73e-02f  1
1394  8.2847667e+01 1.45e+00 4.21e+02  -3.8 8.42e+02    -  1.58e-01 7.53e-02f  1
1395  7.7241397e+01 1.99e+01 5.42e+02  -3.8 6.55e+02    -  1.06e-02 3.46e-01f  1
1396  7.7240498e+01 1.99e+01 4.19e+02  -3.8 5.76e+02    -  4.55e-01 1.09e-04h  1
1397  7.7239393e+01 1.99e+01 4.40e+02  -3.8 6.43e+02    -  1.64e-02 1.22e-04h  1
1398  7.7238448e+01 1.99e+01

1521  7.0779169e+01 4.27e-01 4.48e+01  -5.7 7.13e-02  -1.8 1.00e+00 3.67e-01h  1
1522  7.0778987e+01 3.09e-01 2.25e+01  -5.7 1.31e-01  -2.2 3.99e-01 5.55e-01h  1
1523  7.0778800e+01 2.93e-01 9.01e-01  -5.7 8.15e-02  -1.8 1.00e+00 1.00e+00f  1
1524  7.0778386e+01 5.78e+00 5.36e+01  -5.7 4.65e-01  -2.3 2.68e-01 5.83e-01h  1
1525  7.0777869e+01 4.50e+00 4.55e+00  -5.7 1.98e-01  -1.9 1.00e+00 1.00e+00h  1
1526  7.0777250e+01 6.88e+00 8.44e+01  -5.7 1.49e+00  -2.4 1.44e-01 9.14e-02h  1
1527  7.0777243e+01 6.80e+00 1.62e+02  -5.7 1.85e-01  -1.9 1.00e+00 1.14e-02h  1
1528  7.0776062e+01 6.68e+00 1.12e+02  -5.7 1.04e+03    -  1.30e-01 4.19e-02h  1
1529  7.0775405e+01 5.54e+00 2.13e+02  -5.7 1.78e+02    -  1.00e+00 2.14e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1530  7.0774775e+01 2.47e+00 1.17e+00  -5.7 9.08e+01    -  1.00e+00 1.00e+00h  1
1531  7.0774855e+01 1.07e+00 1.22e+02  -5.7 2.18e-01  -2.4 2.11e-01 8.37e-01h  1
1532  7.0775112e+01 5.02e-01

In [11]:
solve_pyomo(para_truth[7])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 197r 9.1271710e+03 9.76e+04 3.93e+01  -4.5 4.04e+01  -0.0 1.45e-01 2.35e-01f  1
 198r 9.1320396e+03 9.76e+04 3.93e+01  -4.5 1.20e+02  -0.5 5.49e-05 1.33e-02f  1
 199r 9.1320450e+03 9.76e+04 3.93e+01  -4.5 4.55e+01  -0.1 3.15e-08 3.90e-05f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 200r 9.1320452e+03 9.76e+04 1.05e+02  -4.5 1.90e+01   1.3 5.81e-03 3.85e-05f  1
 201r 9.1361990e+03 9.76e+04 1.98e+02  -4.5 6.40e+00   0.8 3.96e-01 2.13e-01f  1
 202r 9.1945263e+03 9.76e+04 3.92e+01  -4.5 1.92e+01   0.3 1.00e+00 1.00e+00f  1
 203r 9.3437882e+03 9.76e+04 3.91e+01  -4.5 5.73e+01  -0.2 7.26e-01 8.55e-01f  1
 204r 9.8637738e+03 9.74e+04 3.85e+01  -4.5 1.69e+02  -0.6 1.00e+00 1.00e+00f  1
 205r 1.0711573e+04 9.71e+04 4.59e+02  -4.5 4.87e+02  -1.1 2.01e-01 5.57e-01f  1
 206r 1.0711704e+04 9.71e+04 4.81e+02  -4.5 1.83e+02  -0.7 1.00e+00 2.29e-04f  1
 207r 1.2394562e+04 9.66e+04 2.20e+02  -4.5 5.33e+02  -1.2 7.50e-01 1.00e+00f  1
 208r 1.3015990e+04 9.64e+04

 506  2.1402837e+05 1.83e+03 4.69e+05  -1.0 1.84e+03  -0.8 3.06e-02 2.04e-03h  1
 507  2.1405728e+05 1.83e+03 4.69e+05  -1.0 1.83e+03  -1.2 6.35e-03 4.58e-03h  1
 508  2.1406711e+05 1.82e+03 5.66e+05  -1.0 1.83e+03  -0.8 3.16e-02 1.49e-03h  1
 509  2.1408320e+05 1.82e+03 5.80e+05  -1.0 1.82e+03  -1.3 2.28e-02 2.63e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 510  2.1411593e+05 1.81e+03 5.75e+05  -1.0 1.82e+03  -1.8 1.43e-02 6.06e-03h  1
 511  2.1412203e+05 1.80e+03 5.90e+05  -1.0 1.81e+03  -1.4 2.27e-02 1.08e-03h  1
 512  2.1416475e+05 1.79e+03 5.86e+05  -1.0 1.80e+03  -1.8 5.19e-03 8.34e-03h  1
 513  2.1416955e+05 1.79e+03 6.30e+05  -1.0 1.79e+03  -1.4 6.66e-02 8.64e-04h  1
 514  2.1420853e+05 1.77e+03 6.25e+05  -1.0 1.79e+03  -1.9 7.65e-03 8.11e-03h  1
 515  2.1422415e+05 1.77e+03 6.42e+05  -1.0 1.77e+03  -1.5 3.53e-02 3.08e-03h  1
 516  2.1425395e+05 1.76e+03 6.38e+05  -1.0 1.77e+03  -1.9 9.12e-03 6.52e-03h  1
 517  2.1427015e+05 1.75e+03

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 620  2.1863195e+05 8.31e+01 1.52e+05  -1.0 4.51e+02  -0.0 5.76e-03 6.03e-04h  1
 621  2.1863536e+05 8.29e+01 1.43e+05  -1.0 4.79e+02  -0.5 4.85e-04 2.75e-03h  1
 622  2.1863858e+05 8.26e+01 1.42e+05  -1.0 4.88e+02  -1.0 1.49e-03 2.56e-03h  1
 623  2.1863931e+05 8.26e+01 1.45e+05  -1.0 4.78e+02  -0.6 1.61e-03 5.55e-04h  1
 624  2.1864138e+05 8.25e+01 1.54e+05  -1.0 5.01e+02  -1.0 8.80e-03 1.53e-03h  1
 625  2.1864789e+05 8.21e+01 1.54e+05  -1.0 5.07e+02  -1.5 5.36e-03 4.58e-03h  1
 626  2.1865406e+05 8.18e+01 1.54e+05  -1.0 5.64e+02  -2.0 1.78e-02 3.59e-03h  1
 627  2.1866312e+05 8.15e+01 1.94e+05  -1.0 6.78e+02  -1.6 5.96e-02 3.80e-03h  1
 628  2.1867765e+05 8.07e+01 1.92e+05  -1.0 1.09e+03  -2.0 8.75e-03 9.37e-03h  1
 629  2.1868065e+05 8.06e+01 2.11e+05  -1.0 9.77e+02  -1.6 1.93e-02 1.62e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 630  2.1869230e+05 7.98e+01

 741  9.2376265e+04 4.68e+02 2.11e+05  -1.0 5.43e+02  -1.4 1.43e-03 5.41e-04h  1
 742  9.2368902e+04 4.68e+02 2.10e+05  -1.0 2.47e+04    -  2.30e-03 1.05e-04f  1
 743  9.2369070e+04 4.68e+02 2.10e+05  -1.0 7.93e+02  -1.9 8.27e-04 6.52e-05h  1
 744  9.2244345e+04 4.67e+02 2.09e+05  -1.0 1.79e+04    -  4.21e-03 2.30e-03f  1
 745  9.2282168e+04 4.58e+02 2.05e+05  -1.0 1.02e+03  -2.4 1.95e-02 1.97e-02h  1
 746  9.1982751e+04 4.56e+02 1.92e+05  -1.0 2.65e+04    -  4.73e-02 4.03e-03f  1
 747  9.1984476e+04 4.56e+02 1.92e+05  -1.0 6.13e+02  -2.0 1.80e-03 4.41e-04h  1
 748  9.1977783e+04 4.56e+02 1.92e+05  -1.0 2.36e+04    -  2.24e-03 9.83e-05f  1
 749  9.1975325e+04 4.49e+02 1.89e+05  -1.0 1.76e+03  -2.4 9.39e-04 1.45e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 750  9.2038875e+04 4.42e+02 1.87e+05  -1.0 6.30e+02  -2.0 1.22e-01 1.60e-02h  1
 751  9.1404551e+04 4.39e+02 1.87e+05  -1.0 2.66e+04    -  4.64e-03 8.16e-03f  1
 752  9.0388678e+04 4.34e+02

 869  8.5416486e+03 4.27e+00 5.36e+03  -1.0 7.52e+03  -3.9 1.67e-03 3.10e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 870  8.5307685e+03 4.27e+00 5.35e+03  -1.0 2.39e+03  -3.5 4.43e-02 1.36e-03f  1
 871  8.1573518e+03 4.23e+00 6.34e+03  -1.0 5.68e+03  -3.9 4.56e-04 1.24e-02f  1
 872  8.1450151e+03 4.22e+00 5.28e+03  -1.0 2.33e+04  -4.4 1.09e-02 1.53e-04f  1
 873  7.5473811e+03 4.06e+00 5.27e+03  -1.0 7.86e+04  -4.9 2.17e-04 2.88e-03f  1
 874  7.4892173e+03 4.06e+00 5.27e+03  -1.0 9.38e+04    -  2.25e-03 2.52e-04f  1
 875  6.7430669e+03 3.85e+00 5.26e+03  -1.0 2.29e+05  -5.4 4.28e-05 1.35e-03f  1
 876  6.7059209e+03 3.85e+00 6.87e+03  -1.0 9.94e+04    -  5.38e-03 1.27e-04f  1
 877  5.5307369e+03 4.66e+00 7.17e+03  -1.0 2.06e+04    -  1.39e-02 1.14e-02f  1
 878  4.4947587e+03 1.56e+01 1.10e+04  -1.0 3.61e+04    -  6.58e-05 5.82e-03f  1
 879  4.4946940e+03 1.56e+01 8.01e+03  -1.0 2.08e-01   2.5 1.21e-01 3.61e-03h  1
iter    objective    inf_pr 

 986  1.2907893e+03 9.61e+00 6.18e+04  -3.8 2.24e+02  -2.7 2.03e-08 3.03e-07f  1
 987  1.2907888e+03 9.61e+00 6.18e+04  -3.8 3.93e+02  -3.2 5.33e-08 2.53e-07f  1
 988  1.2907886e+03 9.61e+00 6.18e+04  -3.8 1.65e+02  -2.7 1.11e-07 2.27e-07f  1
 989  1.2907883e+03 9.61e+00 6.18e+04  -3.8 4.73e+02  -3.2 1.16e-06 1.91e-07f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 990  1.2907882e+03 9.61e+00 5.60e+04  -3.8 1.97e+02  -2.8 2.02e-01 7.95e-07f  1
 991  1.2522959e+03 9.67e+00 5.61e+04  -3.8 9.20e+04    -  5.64e-04 8.97e-04f  1
 992  1.2472660e+03 9.67e+00 5.53e+04  -3.8 1.72e+04    -  4.19e-03 3.29e-04f  1
 993  1.2240345e+03 9.67e+00 5.39e+04  -3.8 1.02e+04    -  9.48e-03 1.73e-03f  1
 994  1.1696680e+03 9.77e+00 5.00e+04  -3.8 7.90e+03    -  4.32e-02 4.95e-03f  1
 995  6.5801766e+02 3.88e+01 5.37e+04  -3.8 5.40e+03    -  1.65e-05 6.41e-02f  1
 996  6.5793392e+02 3.88e+01 5.28e+04  -3.8 2.20e+03    -  1.46e-02 4.38e-05f  1
 997  6.5724696e+02 3.87e+01

1135  9.1553215e+01 1.64e+01 2.75e+03  -3.8 4.10e+02    -  7.01e-02 4.75e-01f  1
1136  9.0933059e+01 1.59e+01 2.61e+03  -3.8 7.30e+02    -  4.80e-02 4.23e-02f  1
1137  9.0501449e+01 1.58e+01 2.62e+03  -3.8 1.09e+02    -  1.57e-04 9.08e-03f  1
1138  9.0494825e+01 1.57e+01 2.29e+03  -3.8 4.69e+02    -  1.22e-01 7.24e-04h  1
1139  9.0050610e+01 1.52e+01 2.29e+03  -3.8 6.72e+01    -  2.63e-05 3.49e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1140  8.9571220e+01 1.17e+01 2.12e+03  -3.8 6.37e+01    -  5.97e-02 2.28e-01h  1
1141  8.9541159e+01 1.15e+01 2.05e+03  -3.8 1.42e+00  -2.3 3.42e-02 1.84e-02h  1
1142  8.9362893e+01 1.14e+01 2.06e+03  -3.8 1.05e+02    -  7.88e-04 1.30e-02f  1
1143  8.9352368e+01 1.13e+01 1.71e+03  -3.8 1.40e+01  -2.8 1.68e-01 3.01e-03h  1
1144  8.9050023e+01 1.12e+01 1.71e+03  -3.8 9.71e+01    -  7.60e-06 1.03e-02f  1
1145  8.8736583e+01 9.81e+00 1.68e+03  -3.8 1.13e+02    -  8.25e-03 1.26e-01h  1
1146  8.8709309e+01 9.72e+00

1254  7.9536669e+01 7.87e+00 6.12e+01  -3.8 2.04e-02  -0.5 6.37e-01 5.50e-01h  1
1255  7.9545038e+01 2.36e-01 7.84e+00  -3.8 4.30e-02  -0.1 1.00e+00 1.00e+00h  1
1256  7.9525326e+01 1.86e-01 4.40e+01  -3.8 3.31e+02    -  1.93e-01 2.35e-01h  1
1257  7.9525528e+01 1.77e-07 7.19e-02  -3.8 2.41e-02  -0.6 1.00e+00 1.00e+00h  1
1258  7.9521657e+01 5.61e-04 3.09e+00  -3.8 1.31e+02    -  1.46e-01 1.38e-01h  1
1259  7.9491999e+01 4.56e-02 5.22e+02  -3.8 3.88e+02    -  7.89e-02 4.65e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1260  7.9496440e+01 1.54e-07 2.40e-01  -3.8 5.51e-02  -1.0 1.00e+00 1.00e+00h  1
1261  7.9495557e+01 2.95e-02 1.41e+02  -3.8 1.87e-01  -1.5 8.80e-01 4.18e-01h  2
1262  7.9487518e+01 3.40e+00 4.89e+02  -3.8 2.16e+03    -  3.60e-02 1.82e-02h  1
1263  7.9488012e+01 1.10e-07 1.39e-01  -3.8 8.80e-03  -0.2 1.00e+00 1.00e+00h  1
1264  7.9475866e+01 3.44e+00 6.78e+01  -3.8 4.77e+02    -  4.57e-01 1.96e-01h  1
1265  7.9476981e+01 8.46e-08

In [12]:
solve_pyomo(para_truth[8])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 187r 9.8905534e+03 9.75e+04 2.21e+03  -3.0 5.14e+02    -  5.24e-02 4.43e-04f  1
 188r 9.9432150e+03 9.75e+04 1.70e+03  -3.0 2.34e+01  -0.7 3.88e-02 4.94e-01f  1
 189r 1.0013338e+04 9.75e+04 1.35e+03  -3.0 6.14e+01  -1.1 3.18e-01 2.44e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 1.0037005e+04 9.75e+04 8.10e+02  -3.0 2.45e+01  -0.7 1.00e+00 2.09e-01f  1
 191r 1.0039277e+04 9.75e+04 8.06e+02  -3.0 6.45e+01  -1.2 8.42e-03 7.38e-03f  1
 192r 1.0164152e+04 9.75e+04 4.66e+00  -3.0 2.69e+01  -0.8 1.00e+00 1.00e+00f  1
 193r 1.0173103e+04 9.75e+04 7.48e+00  -3.0 6.64e+01  -1.2 2.55e-02 2.78e-02f  1
 194r 1.0304468e+04 9.74e+04 4.28e+00  -3.0 2.78e+01  -0.8 1.00e+00 1.00e+00f  1
 195r 1.0343884e+04 9.74e+04 2.17e+02  -3.0 6.76e+01  -1.3 5.01e-02 1.19e-01f  1
 196r 1.0345431e+04 9.74e+04 1.54e+02  -3.0 3.78e+00   0.0 1.00e+00 8.57e-02f  1
 197r 1.0398368e+04 9.74e+04 2.21e+02  -3.0 1.11e+01  -0.4 3.57e-01 1.00e+00f  1
 198r 1.0417882e+04 9.74e+04

 433r 4.2120024e+04 8.99e+04 9.99e+02   5.0 0.00e+00   3.0 0.00e+00 1.01e-08R  2
 434r 4.2111618e+04 8.99e+04 9.99e+02   5.0 6.56e+08    -  4.60e-09 1.20e-08f  1
 435r 4.1855046e+04 9.00e+04 9.99e+02   2.9 9.85e+05    -  1.46e-05 1.45e-04f  1
 436r 4.1984901e+04 9.00e+04 9.99e+02   2.9 3.19e+05    -  1.59e-03 1.09e-04f  1
 437r 4.3525183e+04 8.96e+04 9.97e+02   2.9 2.93e+05    -  1.56e-03 1.37e-03f  1
 438r 4.5084500e+04 8.92e+04 8.82e+03   2.9 3.10e+05    -  1.24e-02 1.30e-03f  1
 439r 4.7699003e+04 8.85e+04 8.37e+03   2.9 5.45e+05    -  9.11e-04 1.25e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 440r 4.9207487e+04 8.81e+04 1.07e+04   2.9 5.64e+05    -  2.38e-03 6.95e-04f  1
 441r 5.0125751e+04 8.78e+04 1.27e+04   2.9 4.33e+05    -  1.65e-03 5.54e-04f  1
 442r 5.2384985e+04 8.72e+04 1.14e+04   2.9 3.37e+05    -  1.28e-03 1.77e-03f  1
 443r 5.2551426e+04 8.72e+04 1.15e+04   2.9 8.10e+05    -  8.11e-05 5.04e-05f  1
 444r 5.3622594e+04 8.69e+04

 732r 1.4087340e+05 6.53e+04 9.99e+02   4.8 0.00e+00   0.7 0.00e+00 3.13e-07R  3
 733r 1.4086782e+05 6.53e+04 9.99e+02   4.8 2.65e+07    -  8.18e-07 1.92e-07f  1
 734r 1.4138183e+05 6.51e+04 1.16e+04   2.7 1.26e+06    -  1.02e-02 9.76e-05f  1
 735r 1.5570035e+05 6.19e+04 1.10e+04   2.7 1.94e+06    -  1.41e-03 1.69e-03f  1
 736r 1.5743830e+05 6.15e+04 1.31e+04   2.7 2.25e+06    -  1.11e-03 1.72e-04f  1
 737r 1.6859310e+05 5.90e+04 1.09e+04   2.7 2.41e+06    -  2.77e-04 1.01e-03f  1
 738r 1.8365708e+05 5.58e+04 1.25e+04   2.7 2.67e+06    -  1.90e-03 1.22e-03f  1
 739  1.8366699e+05 5.58e+04 5.87e+03  -1.0 5.58e+04   0.2 6.69e-05 1.50e-05h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 740  1.8367554e+05 5.58e+04 5.87e+03  -1.0 5.58e+04  -0.3 6.59e-05 1.42e-05h  1
 741r 1.8367554e+05 5.58e+04 9.99e+02   4.7 0.00e+00  -0.8 0.00e+00 2.96e-07R  6
 742r 1.8366541e+05 5.58e+04 1.08e+03   4.7 1.59e+08    -  1.59e-06 1.64e-07f  1
 743r 1.8571731e+05 5.53e+04

 857  3.3395090e+05 1.79e+04 1.12e+06  -1.0 3.47e+04  -2.9 8.89e-02 2.42e-04h  1
 858  3.3424450e+05 1.78e+04 1.32e+06  -1.0 3.57e+04  -2.4 3.88e-02 5.33e-03h  1
 859  3.3567140e+05 1.73e+04 1.27e+06  -1.0 3.60e+04  -2.9 1.49e-03 2.68e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 860  3.3572888e+05 1.73e+04 1.27e+06  -1.0 3.44e+04  -2.5 1.45e-03 1.14e-03h  1
 861  3.3587316e+05 1.72e+04 1.28e+06  -1.0 3.46e+04  -3.0 5.25e-02 2.90e-03h  1
 862  3.3692612e+05 1.69e+04 1.19e+06  -1.0 3.43e+04  -2.5 5.33e-03 2.11e-02h  1
 863  3.3695335e+05 1.69e+04 1.19e+06  -1.0 3.50e+04  -3.0 3.02e-02 5.42e-04h  1
 864  3.3730540e+05 1.67e+04 1.27e+06  -1.0 3.53e+04  -2.6 3.37e-02 6.71e-03h  1
 865  3.3893924e+05 1.62e+04 1.24e+06  -1.0 3.56e+04  -3.1 1.95e-03 3.33e-02h  1
 866  3.3906944e+05 1.61e+04 1.23e+06  -1.0 3.41e+04  -2.6 6.92e-04 2.66e-03h  1
 867  3.4056910e+05 1.56e+04 1.16e+06  -1.0 3.51e+04  -3.1 7.86e-02 3.19e-02h  1
 868  3.4058607e+05 1.56e+04

...

2559  9.0538857e+02 1.20e+01 1.21e+04  -3.8 3.36e+03    -  1.36e-02 2.41e-03f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2560  7.2028993e+02 2.99e+01 1.12e+04  -3.8 2.43e+03    -  4.08e-02 2.98e-02f  1
2561  7.2023008e+02 2.99e+01 1.12e+04  -3.8 2.21e+03    -  6.04e-03 3.60e-05f  1
2562  7.0615028e+02 2.96e+01 1.10e+04  -3.8 3.26e+03    -  1.85e-06 9.31e-03f  1
2563  7.0614678e+02 2.96e+01 1.15e+04  -3.8 3.58e+03    -  4.44e-02 3.09e-06h  1
2564  7.0094118e+02 2.95e+01 1.14e+04  -3.8 3.73e+03    -  8.39e-06 4.99e-03f  1
2565  6.9169911e+02 2.93e+01 1.14e+04  -3.8 2.87e+03    -  2.94e-02 5.38e-03f  1
2566  6.9140220e+02 2.93e+01 1.14e+04  -3.8 3.13e+03    -  3.24e-05 2.16e-04f  1
2567  6.7982250e+02 2.91e+01 1.14e+04  -3.8 2.96e+03    -  1.66e-02 7.76e-03f  1
2568  6.7975694e+02 2.91e+01 1.13e+04  -3.8 4.56e+03    -  1.30e-05 7.88e-05f  1
2569  6.7823393e+02 2.91e+01 1.12e+04  -3.8 1.50e+05    -  4.48e-05 1.41e-04f  1
iter    objective    inf_pr 

2673  3.2452937e+02 8.24e-01 7.04e+03  -3.8 5.75e+01  -2.6 2.75e-05 1.28e-03f  1
2674  3.2380192e+02 7.49e-01 6.51e+03  -3.8 1.18e+01  -2.2 8.70e-02 7.84e-02f  1
2675  3.2351115e+02 7.09e-01 6.61e+03  -3.8 3.98e+00  -1.8 2.55e-01 5.37e-02f  1
2676  3.2157954e+02 5.15e-01 4.56e+03  -3.8 8.55e+00  -2.3 2.03e-01 2.78e-01f  1
2677  3.2109240e+02 4.96e-01 4.33e+03  -3.8 2.30e+01  -2.7 1.15e-02 3.57e-02f  1
2678  3.2050634e+02 4.87e-01 4.35e+03  -3.8 6.76e+01  -3.2 5.02e-02 1.75e-02f  1
2679  3.1940502e+02 3.36e-01 3.35e+03  -3.8 3.87e+00  -1.9 4.42e-01 2.91e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2680  3.1870842e+02 2.94e-01 3.62e+03  -3.8 8.93e+00  -2.4 6.06e-01 1.27e-01f  1
2681  3.1870618e+02 2.94e-01 3.62e+03  -3.8 2.66e+01  -2.9 3.44e-03 1.71e-04h  1
2682  3.1848976e+02 2.92e-01 3.57e+03  -3.8 7.89e+01  -3.3 5.20e-02 6.05e-03f  1
2683  3.1771485e+02 2.90e-01 3.55e+03  -3.8 2.02e+02  -3.8 7.91e-03 8.56e-03f  1
2684  3.1400235e+02 2.97e-01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2790  1.1710446e+02 4.37e+00 6.97e+03  -3.8 4.02e+03    -  5.03e-09 3.18e-07f  1
2791  1.1710441e+02 4.37e+00 6.97e+03  -3.8 2.77e+03    -  1.05e-08 1.60e-07f  2
2792  1.1710441e+02 4.37e+00 7.51e+03  -3.8 1.75e+00   1.2 6.34e-03 7.45e-07h  3
2793  1.1700895e+02 4.33e+00 3.24e+03  -3.8 1.52e+02    -  5.49e-01 7.47e-03h  1
2794  1.1547896e+02 2.60e+00 3.52e+03  -3.8 6.13e+01    -  1.21e-04 4.01e-01f  1
2795  1.1549135e+02 2.48e+00 3.83e+03  -3.8 2.58e-01   1.6 6.25e-02 4.57e-02h  1
2796  1.1550064e+02 2.37e+00 1.03e+05  -3.8 2.62e-01   2.1 1.00e+00 4.17e-02h  1
2797  1.1553941e+02 2.06e+00 9.20e+04  -3.8 3.06e-01   1.6 8.46e-02 1.33e-01h  1
2798  1.1559214e+02 1.69e+00 7.89e+03  -3.8 2.82e-01   1.1 9.60e-01 1.77e-01h  1
2799  1.1566744e+02 1.17e+00 3.88e+02  -3.8 2.39e-01   0.6 6.15e-01 3.08e-01h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2800  1.1577181e+02 4.34e-01

2903  7.7370819e+01 5.17e-03 2.88e+01  -5.7 5.49e-02  -0.5 2.02e-03 1.80e-02f  1
2904  7.7370822e+01 5.17e-03 2.76e+01  -5.7 1.30e+00  -0.9 2.73e-06 2.75e-04f  2
2905  7.7370641e+01 1.50e-04 1.22e+00  -5.7 5.55e-03   0.4 1.00e+00 9.46e-01h  1
2906  7.7317735e+01 1.14e-03 1.05e+02  -5.7 2.71e+02    -  5.20e-03 5.49e-02f  1
2907  7.7317912e+01 5.52e-04 3.15e+01  -5.7 4.10e-02  -0.1 7.03e-01 8.98e-01h  1
2908  7.7317751e+01 1.65e-04 2.42e+00  -5.7 7.62e-03   0.3 1.00e+00 8.73e-01f  1
2909  7.7317614e+01 9.28e-09 1.77e-02  -5.7 3.06e-03   0.8 1.00e+00 1.00e+00f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
2910  7.7317555e+01 1.08e-08 2.50e+01  -5.7 1.38e-02   0.3 1.00e+00 1.18e-01h  1
2911  7.7316493e+01 6.22e-08 2.98e+01  -5.7 1.44e+02    -  8.73e-03 2.07e-03f  1
2912  7.7316467e+01 1.16e-08 1.52e-02  -5.7 2.36e-02  -0.2 1.00e+00 1.00e+00f  1
2913  7.7315583e+01 1.04e-07 5.74e-01  -5.7 2.49e+01    -  7.16e-03 1.01e-02f  1
2914  7.7315583e+01 6.94e-08

3155  7.5335572e+01 6.68e-04 4.45e-02  -8.6 4.23e-04  -0.7 1.00e+00 2.66e-01h  1
3156  7.5335572e+01 5.84e-04 2.68e-02  -8.6 6.68e-05  -1.1 1.00e+00 1.25e-01f  4
3157  7.5335572e+01 1.46e-11 6.97e-06  -8.6 2.80e-04  -1.6 1.00e+00 1.00e+00h  1
3158  7.5335572e+01 1.46e-11 4.11e-06  -8.6 4.96e-04  -2.1 1.00e+00 1.00e+00h  1
3159  7.5335572e+01 1.46e-11 3.84e-06  -8.6 1.39e-03  -2.6 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
3160  7.5335572e+01 6.37e-11 3.75e-06  -8.6 4.08e-03  -3.0 1.00e+00 1.00e+00h  1
3161  7.5335570e+01 5.50e-10 3.74e-06  -8.6 1.22e-02  -3.5 1.00e+00 1.00e+00h  1
3162  7.5335566e+01 3.70e-09 7.10e-03  -8.6 3.65e-02  -4.0 1.00e+00 8.57e-01h  1
3163  7.5335564e+01 3.75e-09 1.23e-01  -8.6 1.15e-01  -4.5 1.00e+00 9.39e-02f  1
3164  7.5335558e+01 1.43e-08 3.01e-01  -8.6 2.68e-01  -4.9 1.00e+00 1.70e-01f  1
3165  7.5335539e+01 8.52e-08 8.98e-01  -8.6 7.94e-01  -5.4 1.00e+00 1.64e-01f  1
3166  7.5335421e+01 1.64e-04

In [13]:
solve_pyomo(para_truth[9])

Ipopt 3.14.6: max_iter=50000


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.6, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:     2256
Number of nonzeros in inequality constraint Jacobian.:      500
Number of nonzeros in Lagrangian Hessian.............:     2100

Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating objective gradient at user provided starting point.
  No scaling factor for objective function computed!
Error in an AMPL evaluation. Run with "halt_on_ampl_error yes" to see details.
Error evaluating Jacobian of equality c

 185r 1.8840052e+04 9.50e+04 4.18e+03  -6.7 1.62e+03  -1.6 9.63e-06 2.29e-01f  1
 186r 1.8840053e+04 9.50e+04 2.69e+03  -6.7 6.09e+02  -1.1 3.09e-01 7.48e-07f  1
 187r 1.9106957e+04 9.49e+04 2.55e+03  -6.7 1.81e+03  -1.6 2.15e-07 4.96e-02f  1
 188r 2.0335718e+04 9.45e+04 9.33e+02  -6.7 6.83e+02  -1.2 6.23e-01 6.08e-01f  1
 189r 2.0973349e+04 9.43e+04 3.79e+02  -6.7 2.56e+02  -0.8 6.13e-01 8.41e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 190r 2.1041415e+04 9.43e+04 1.47e+03  -6.7 7.65e+02  -1.2 7.65e-01 3.01e-02f  1
 191r 2.1652200e+04 9.41e+04 2.89e+03  -6.7 2.28e+03  -1.7 5.86e-01 9.04e-02f  1
 192r 2.2024142e+04 9.39e+04 1.95e+03  -6.7 8.59e+02  -1.3 9.02e-01 1.46e-01f  1
 193r 2.2024341e+04 9.39e+04 3.53e+03  -6.7 2.55e+03  -1.8 5.16e-01 2.63e-05f  1
 194r 2.4653221e+04 9.31e+04 2.12e+02  -6.7 9.65e+02  -1.3 9.99e-01 9.21e-01f  1
 195r 2.4710159e+04 9.30e+04 2.77e+02  -6.7 2.85e+03  -1.8 1.17e-04 6.72e-03f  1
 196r 2.6429624e+04 9.25e+04

 428r 1.7986062e+05 1.67e+04 9.99e+02   4.2 0.00e+00  -0.2 0.00e+00 3.96e-07R  5
 429r 1.7985889e+05 1.67e+04 2.65e+04   4.2 5.00e+07    -  7.62e-05 8.14e-09f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 430r 1.8603772e+05 1.30e+04 1.19e+04   2.8 1.31e+06    -  1.29e-03 2.89e-03f  1
 431  1.8605267e+05 1.30e+04 4.63e+03  -1.0 1.30e+04   0.2 5.31e-05 9.04e-05h  1
 432  1.8607541e+05 1.30e+04 4.71e+03  -1.0 1.30e+04  -0.3 2.44e-04 1.56e-04h  1
 433r 1.8607541e+05 1.30e+04 9.99e+02   4.1 0.00e+00  -0.8 0.00e+00 4.34e-07R  4
 434r 1.8607210e+05 1.30e+04 9.99e+02   4.1 9.44e+06    -  4.40e-07 1.81e-07f  1
 435r 1.8618595e+05 1.30e+04 1.84e+04   2.7 1.69e+06    -  3.23e-03 3.86e-05f  1
 436r 1.9413297e+05 9.10e+03 2.16e+04   2.7 1.70e+06    -  3.01e-03 2.33e-03f  1
 437  1.9413699e+05 9.10e+03 5.21e+03  -1.0 9.10e+03  -0.3 1.94e-04 3.66e-05h  1
 438  1.9413942e+05 9.10e+03 5.60e+03  -1.0 9.10e+03   0.1 3.20e-04 2.13e-05h  1
 439  1.9414867e+05 9.10e+03

 538  2.1429890e+05 7.27e+01 3.69e+05  -1.0 1.22e+03  -1.4 2.50e-03 1.87e-03f  1
 539  2.1429735e+05 7.26e+01 4.36e+05  -1.0 1.21e+03  -0.9 2.68e-02 1.75e-03h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 540  2.1429504e+05 7.24e+01 4.35e+05  -1.0 1.22e+03  -1.4 2.30e-03 2.40e-03f  1
 541  2.1429482e+05 7.24e+01 7.81e+05  -1.0 1.22e+03  -1.0 9.47e-02 4.03e-04h  1
 542  2.1429571e+05 7.23e+01 8.21e+05  -1.0 1.89e+03  -1.5 1.14e-02 1.08e-03h  1
 543  2.1429717e+05 7.22e+01 9.38e+05  -1.0 1.64e+03  -1.0 1.46e-02 2.23e-03h  1
 544  2.1429950e+05 7.19e+01 9.23e+05  -1.0 2.04e+03  -1.5 9.63e-04 3.70e-03h  1
 545  2.1429959e+05 7.19e+01 1.45e+06  -1.0 1.78e+03  -1.1 5.11e-02 1.52e-04h  1
 546  2.1430218e+05 7.15e+01 1.55e+06  -1.0 2.21e+03  -1.6 2.87e-02 4.75e-03h  1
 547  2.1430327e+05 7.13e+01 1.56e+06  -1.0 2.00e+03  -2.0 1.83e-02 3.06e-03h  1
 548  2.1430362e+05 7.12e+01 1.55e+06  -1.0 1.92e+03  -1.6 3.85e-04 1.58e-03h  1
 549  2.1430386e+05 7.10e+01

 644r 2.1355606e+05 1.70e+01 9.99e+02   1.2 0.00e+00   1.8 0.00e+00 3.57e-07R  5
 645r 2.1368118e+05 1.24e+01 4.90e+03   1.2 9.19e+04    -  6.29e-03 1.64e-03f  1
 646  2.1368123e+05 1.24e+01 5.25e+03  -1.0 7.42e+01   1.4 5.31e-05 3.74e-05h  1
 647  2.1368129e+05 1.24e+01 5.25e+03  -1.0 8.03e+01   0.9 1.25e-04 4.33e-05f  1
 648  2.1368143e+05 1.24e+01 5.26e+03  -1.0 1.01e+02   0.4 2.85e-04 1.44e-04h  1
 649  2.1368150e+05 1.24e+01 5.27e+03  -1.0 1.61e+02  -0.1 1.09e-03 2.30e-04f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 650  2.1368144e+05 1.23e+01 5.32e+03  -1.0 1.33e+02  -0.6 6.72e-03 8.04e-04f  1
 651  2.1368124e+05 1.23e+01 5.37e+03  -1.0 1.83e+02  -1.0 4.30e-03 5.67e-04f  1
 652  2.1368086e+05 1.23e+01 5.74e+03  -1.0 2.31e+02  -1.5 3.03e-02 7.21e-04f  1
 653  2.1367889e+05 1.23e+01 2.31e+04  -1.0 2.31e+02  -2.0 7.46e-02 3.11e-03f  1
 654  2.1366208e+05 1.20e+01 1.68e+04  -1.0 5.34e+02  -2.5 1.51e-02 2.35e-02f  1
 655  2.1366203e+05 1.20e+01

 758  1.4983325e+05 4.24e+01 2.20e+04  -1.0 2.13e+03  -3.3 4.60e-03 3.65e-02f  1
 759  1.4972206e+05 4.02e+01 7.39e+03  -1.0 8.16e+02  -2.9 4.10e-01 5.00e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 760  1.4969089e+05 4.00e+01 7.73e+03  -1.0 1.77e+03  -3.4 1.82e-02 5.92e-03f  1
 761  1.4947659e+05 3.95e+01 9.55e+03  -1.0 5.07e+03  -3.8 5.37e-02 1.35e-02f  1
 762  1.4947973e+05 3.91e+01 8.88e+03  -1.0 1.20e+01   1.1 2.35e-03 1.00e-02h  1
 763  1.4955691e+05 3.01e+01 3.57e+04  -1.0 1.94e+00   1.5 4.33e-03 2.29e-01h  1
 764  1.4981670e+05 3.00e-01 5.87e+04  -1.0 3.50e-01   1.0 2.90e-01 9.90e-01h  1
 765  1.4981672e+05 1.51e-01 3.62e+04  -1.0 4.06e-01   0.6 9.92e-01 4.97e-01f  2
 766  1.4981343e+05 2.75e-02 3.28e+04  -1.0 1.22e+00   0.1 1.00e+00 8.18e-01f  1
 767  1.4980077e+05 1.07e-03 1.54e+00  -1.0 3.74e+00  -0.4 1.00e+00 1.00e+00f  1
 768  1.4976370e+05 1.01e-02 6.35e+05  -3.8 1.15e+01  -0.9 1.88e-01 8.67e-01f  1
 769  1.4970805e+05 2.77e-02

 863  1.0457395e+05 9.14e+01 1.14e+04  -3.8 2.61e+04  -4.6 3.28e-02 2.73e-05f  1
 864  1.0057938e+05 9.64e+01 1.12e+04  -3.8 7.86e+04  -5.1 1.55e-02 1.88e-02f  1
 865  1.0057860e+05 9.64e+01 1.12e+04  -3.8 2.95e+04  -4.7 1.69e-01 9.74e-06f  1
 866  1.0057348e+05 9.64e+01 1.13e+04  -3.8 8.56e+04  -5.2 4.31e-02 2.17e-05f  1
 867  9.9713001e+04 9.58e+01 1.11e+04  -3.8 3.18e+04  -4.7 3.53e-05 9.80e-03f  1
 868  9.9707762e+04 9.58e+01 1.27e+04  -3.8 1.22e+04  -4.3 2.85e-01 1.53e-04f  1
 869  9.7444329e+04 9.61e+01 1.23e+04  -3.8 3.59e+04  -4.8 4.36e-05 2.28e-02f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 870  9.7333301e+04 9.57e+01 1.23e+04  -3.8 1.79e+05  -5.3 8.93e-05 3.10e-04f  1
 871  9.7332112e+04 9.57e+01 1.23e+04  -3.8 5.80e+04  -4.9 3.98e-06 9.25e-06f  1
 872  9.7317566e+04 9.57e+01 1.34e+04  -3.8 1.54e+04  -4.4 2.60e-01 3.39e-04f  1
 873  9.5112852e+04 9.60e+01 1.32e+04  -3.8 4.58e+04  -4.9 4.21e-05 1.72e-02f  1
 874  9.5020786e+04 9.59e+01

 976  4.5106042e+04 6.13e+01 3.48e+03  -3.8 4.91e+00   0.2 7.16e-03 2.12e-02h  1
 977  4.5124211e+04 5.89e+01 3.33e+03  -3.8 4.91e+00  -0.3 2.85e-02 3.92e-02h  1
 978  4.5125008e+04 5.88e+01 3.46e+03  -3.8 9.33e+00  -0.7 9.99e-02 1.86e-03h  1
 979  4.5142051e+04 5.61e+01 3.29e+03  -3.8 2.77e+01  -1.2 2.39e-01 4.57e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 980  4.5148928e+04 5.41e+01 3.20e+03  -3.8 8.20e+01  -1.7 4.96e-02 3.48e-02h  1
 981  4.5142752e+04 5.30e+01 3.40e+03  -3.8 2.40e+02  -2.2 1.12e-01 2.08e-02f  1
 982  4.5142604e+04 5.30e+01 3.43e+03  -3.8 4.81e+02  -2.6 1.20e-02 1.32e-04f  1
 983  4.5125289e+04 5.27e+01 3.47e+03  -3.8 1.35e+03  -3.1 1.73e-02 4.66e-03f  1
 984  4.5103430e+04 5.26e+01 3.47e+03  -3.8 3.66e+03  -3.6 3.43e-03 2.21e-03f  1
 985  4.5101357e+04 5.26e+01 3.47e+03  -3.8 8.96e+03  -4.1 7.71e-04 9.28e-05f  1
 986  4.5101355e+04 5.26e+01 3.50e+03  -3.8 4.64e+03  -3.6 5.94e-03 1.82e-07h  1
 987  4.5072973e+04 5.25e+01

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1110  7.5238726e+02 9.28e-01 2.33e+02  -3.8 1.93e+02    -  5.20e-02 5.51e-02f  1
1111  6.7472775e+02 9.54e-01 1.95e+02  -3.8 1.83e+02    -  8.61e-02 1.12e-01f  1
1112  6.5105276e+02 9.30e-01 2.35e+02  -3.8 1.61e+02    -  1.26e-01 3.85e-02f  1
1113  5.8737283e+02 9.17e-01 2.52e+02  -3.8 1.54e+02    -  1.85e-01 1.08e-01f  1
1114  5.1620087e+02 9.00e-01 2.73e+02  -3.8 1.36e+02    -  2.26e-01 1.36e-01f  1
1115  4.6629353e+02 8.53e-01 3.17e+02  -3.8 1.18e+02    -  2.16e-01 1.10e-01f  1
1116  4.3363953e+02 8.05e-01 4.04e+02  -3.8 1.21e+02    -  2.27e-01 8.12e-02f  1
1117  3.8537872e+02 7.43e-01 4.08e+02  -3.8 1.46e+02    -  2.00e-01 1.31e-01f  1
1118  3.6873375e+02 7.10e-01 3.77e+02  -3.8 1.45e+02    -  4.16e-02 5.22e-02f  1
1119  3.3736507e+02 6.55e-01 4.12e+02  -3.8 1.34e+02    -  1.80e-01 1.04e-01f  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1120  3.0700210e+02 5.98e-01

1243  7.5034597e+01 1.11e-03 2.16e+01  -3.8 1.13e+02    -  9.80e-01 2.50e-01h  3
1244  7.5043985e+01 1.91e-04 1.79e+01  -3.8 2.39e+02    -  8.75e-01 1.00e+00H  1
1245  7.5044872e+01 1.21e-07 5.37e-02  -3.8 1.60e-02  -1.6 1.00e+00 1.00e+00h  1
1246  7.5041440e+01 1.96e-02 1.23e+00  -3.8 3.63e+02    -  2.32e-01 2.17e-01h  1
1247  7.5035007e+01 2.53e-03 3.52e+01  -3.8 2.66e+02    -  1.00e+00 6.43e-01F  1
1248  7.5037495e+01 1.22e-03 1.78e-01  -3.8 6.97e+01    -  1.00e+00 1.00e+00f  1
1249  7.5036504e+01 5.28e-04 6.24e-02  -3.8 3.71e+01    -  1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
1250  7.5036192e+01 6.42e-06 5.43e-03  -3.8 1.11e+01    -  1.00e+00 1.00e+00h  1
1251  7.5036159e+01 1.96e-07 7.11e-05  -3.8 1.20e+00    -  1.00e+00 1.00e+00h  1
1252  7.5024289e+01 1.57e-04 3.35e+03  -5.7 6.07e+01    -  5.44e-01 3.18e-01f  1
1253  7.5009417e+01 3.57e-05 1.04e+02  -5.7 4.76e-02  -1.2 6.45e-01 7.73e-01h  1
1254  7.5004357e+01 4.79e-01